<a href="https://colab.research.google.com/github/phylypo/khmer-language-model-ulmfit/blob/master/ULMFiT_for_Khmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ULMFiT from scratch

### Setup

In [2]:
# mount google drive, zip up the data and copy over
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [0]:
bs=48
# bs=24

In [0]:
torch.cuda.set_device(0)

In [0]:
data_path = Config.data_path()

In [0]:
# km wikipedia has 7600 articles --only got 2487
lang = 'km'

In [0]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [18]:
import urllib.request
url = "https://raw.githubusercontent.com/fastai/course-nlp/master/nlputils.py"
print("Downloading from:", url)
urllib.request.urlretrieve(url, "nlputils.py")

('nlputils.py', <http.client.HTTPMessage at 0x7fe19a6b55c0>)

### Install CRF

In [0]:
# Character `!` in the beginning of the line indicates a shell command
!pip install sklearn_crfsuite

# import of the crf library
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

     |████████████████████████████████| 757kB 7.2MB/s 


In [0]:
import urllib.request
file = 'sklearn_crf_model_90k-100i.sav'
url = 'https://github.com/phylypo/segmentation-crf-khmer/blob/master/models/sklearn_crf_model_90k-100i.sav?raw=true'
#file = 'sklearn_crf_model_100-100i.sav'
#url = 'https://github.com/phylypo/segmentation-crf-khmer/blob/master/models/sklearn_crf_model_100-100i.sav?raw=true'
print("new url", url)
urllib.request.urlretrieve(url, file)

new url https://github.com/phylypo/segmentation-crf-khmer/blob/master/models/sklearn_crf_model_90k-100i.sav?raw=true


('sklearn_crf_model_90k-100i.sav', <http.client.HTTPMessage at 0x7ff42ad584a8>)

### CRF Setup

In [0]:
#@title Define KCC and feature generation
# consonant and independent vowels
KHCONST = set(u'កខគឃងចឆជឈញដឋឌឍណតថទធនបផពភមយរលវឝឞសហឡអឣឤឥឦឧឨឩឪឫឬឭឮឯឰឱឲឳ')
KHVOWEL = set(u'឴឵ាិីឹឺុូួើឿៀេែៃោៅ\u17c6\u17c7\u17c8')
# subscript, diacritics
KHSUB = set(u'្')
KHDIAC = set(u"\u17c9\u17ca\u17cb\u17cc\u17cd\u17ce\u17cf\u17d0") #MUUSIKATOAN, TRIISAP, BANTOC,ROBAT,
KHSYM = set('៕។៛ៗ៚៙៘')
KHNUMBER = set(u'០១២៣៤៥៦៧៨៩0123456789') # remove 0123456789
# lunar date:  U+19E0 to U+19FF ᧠...᧿
KHLUNAR = set('᧠᧡᧢᧣᧤᧥᧦᧧᧨᧩᧪᧫᧬᧭᧮᧯᧰᧱᧲᧳᧴᧵᧶᧷᧸᧹᧺᧻᧼᧽᧾᧿')

def is_khmer_char(ch):
  if (ch >= '\u1780') and (ch <= '\u17ff'): return True
  if ch in KHLUNAR: return True
  return False

def is_start_of_kcc(ch):
  if is_khmer_char(ch):
    if ch in KHCONST: return True
    if ch in KHSYM: return True
    if ch in KHNUMBER: return True
    if ch in KHLUNAR: return True
    return False
  return True

# kcc base
def seg_kcc(str_sentence):
    str_sentence = str_sentence.replace(u'\u200b','')
    segs = []
    cur = ""
    for phr in str_sentence.split(' '):
        #print("PHR:[%s] len:%d" %(phr, len(phr)))
        for i,c in enumerate(phr):
            #print(i," c:", c)
            cur += c
            nextchar = phr[i+1] if (i+1 < len(phr)) else ""
            
            # cluster non-khmer chars together
            if not is_khmer_char(c) and nextchar != "" and not is_khmer_char(nextchar): continue
            # cluster number together
            if c in KHNUMBER and nextchar in KHNUMBER: continue
              
            # cluster non-khmer together
            # non-khmer character has no cluster
            if not is_khmer_char(c) or nextchar=="":
                segs.append(cur)
                cur=""
            elif is_start_of_kcc(nextchar) and not (c in KHSUB):
                segs.append(cur)
                cur=""          
    return segs

# testing some text
t1 = "យោងតាមប្រភពព័ត៌មានបានឱ្យដឹងថា កាលពីពេលថ្មីៗនេះក្រុមចក្រភពអង់គ្លេស Royal Marines ដែលមានមូលដ្ឋាននៅ Gibraltar បានរឹបអូសយកនាវាដឹកប្រេងឆៅរបស់អ៊ីរ៉ង់ដែលធ្វើដំណើរទៅកាន់រោងចក្រចម្រាញ់ប្រេងនៅក្នុងប្រទេសស៊ីរី ដោយក្រុងឡុងដ៍អះអាងថា ការរឹបអូសត្រូវបានគេសំដៅអនុវត្ត៕"
t2 = "This is a test."
t3 = "នៅរសៀលថ្ងៃទី២២ ខែ កក្កដា ឆ្នាំ២០១៩ ឯកឧត្តម គួច ចំរើន អភិបាលខេត្តព្រះសីហនុ"
print("kcc:", seg_kcc(t1))

kcc: ['យោ', 'ង', 'តា', 'ម', 'ប្រ', 'ភ', 'ព', 'ព័', 'ត៌', 'មា', 'ន', 'បា', 'ន', 'ឱ្យ', 'ដឹ', 'ង', 'ថា', 'កា', 'ល', 'ពី', 'ពេ', 'ល', 'ថ្មី', 'ៗ', 'នេះ', 'ក្រុ', 'ម', 'ច', 'ក្រ', 'ភ', 'ព', 'អ', 'ង់', 'គ្លេ', 'ស', 'Royal', 'Marines', 'ដែ', 'ល', 'មា', 'ន', 'មូ', 'ល', 'ដ្ឋា', 'ន', 'នៅ', 'Gibraltar', 'បា', 'ន', 'រឹ', 'ប', 'អូ', 'ស', 'យ', 'ក', 'នា', 'វា', 'ដឹ', 'ក', 'ប្រេ', 'ង', 'ឆៅ', 'រ', 'ប', 'ស់', 'អ៊ី', 'រ៉', 'ង់', 'ដែ', 'ល', 'ធ្វើ', 'ដំ', 'ណើ', 'រ', 'ទៅ', 'កា', 'ន់', 'រោ', 'ង', 'ច', 'ក្រ', 'ច', 'ម្រា', 'ញ់', 'ប្រេ', 'ង', 'នៅ', 'ក្នុ', 'ង', 'ប្រ', 'ទេ', 'ស', 'ស៊ី', 'រី', 'ដោ', 'យ', 'ក្រុ', 'ង', 'ឡុ', 'ង', 'ដ៍', 'អះ', 'អា', 'ង', 'ថា', 'កា', 'រ', 'រឹ', 'ប', 'អូ', 'ស', 'ត្រូ', 'វ', 'បា', 'ន', 'គេ', 'សំ', 'ដៅ', 'អ', 'នុ', 'វ', 'ត្ត', '៕']


In [0]:
# generate list of (word, label), not splitting into phrases, just remove spaces
def gen_kcc_with_label(sentence):
    words = sentence.split()
    final_kccs = []
    for word in words:
        kccs = seg_kcc(word)
        labels = [1 if (i==0) else 0 for i, k in enumerate(kccs)]
        final_kccs.extend(list(zip(kccs,labels)))
    return final_kccs

# test label
ts = "This is a test"
ts = "នៅ រសៀល ថ្ងៃ ទី ២២ ខែ កក្កដា ឆ្នាំ ២០១៩ ។"
kccs = seg_kcc(ts)
kl = gen_kcc_with_label(ts)
print("len of kcc:", len(kccs), " data:", kccs)
print("kcc with label len:", len(kl), " data:", kl)

len of kcc: 14  data: ['នៅ', 'រ', 'សៀ', 'ល', 'ថ្ងៃ', 'ទី', '២២', 'ខែ', 'ក', 'ក្ក', 'ដា', 'ឆ្នាំ', '២០១៩', '។']
kcc with label len: 14  data: [('នៅ', 1), ('រ', 1), ('សៀ', 0), ('ល', 0), ('ថ្ងៃ', 1), ('ទី', 1), ('២២', 1), ('ខែ', 1), ('ក', 1), ('ក្ក', 0), ('ដា', 0), ('ឆ្នាំ', 1), ('២០១៩', 1), ('។', 1)]


In [0]:
#@title Define char type

EN = set(u'abcdefghijklmnopqrstuvwxyz0123456789')

# E=English, C=Consonant, W=wowel, N=number, O=Other, S=subcript, D=Diacritic, NS=no_space
# roll up to: NS, C, W, S, D
NS = 'NS'
def get_type(chr):
  if chr.lower() in EN: return "NS"
  if chr in KHCONST: return "C"
  if chr in KHVOWEL: return "W"
  if chr in KHNUMBER: return "NS"
  if chr in KHSUB: return "S"
  if chr in KHDIAC: return "D"
  return "NS" #not U - same as N

# non-khmer character that we should not separate like number
# multiple characters are false
def is_no_space(k):
  if len(k) > 1: return False
  if get_type(k)==NS: return True
  return False

def kcc_type(k):
  if len(k)==1: return get_type(k)
  else: return "K" + str(len(k))

print("kcc_type for u'\u17cb'", kcc_type(u'\u17cb'))
print("kcc_type for u'A'", kcc_type('A'))
print("kcc_type for u'២'", kcc_type('២'))
print("kcc_type for 'ថ្ងៃ'", kcc_type('ថ្ងៃ'))
print("is NS for 'A':", is_no_space("A"))
print("is NS for 'ថ្ងៃ':", is_no_space('ថ្ងៃ'))

kcc_type for u'់' D
kcc_type for u'A' NS
kcc_type for u'២' NS
kcc_type for 'ថ្ងៃ' K4
is NS for 'A': True
is NS for 'ថ្ងៃ': False


In [0]:
#@title Define CRF features
# only pass in kccs list (without labels)
def kcc_to_features(kccs, i):
    maxi = len(kccs)
    kcc = kccs[i]

    features = {
        #'bias': 1.0,
        'kcc': kcc,
        't': kcc_type(kcc),
        'ns': is_no_space(kcc)
    }
    if i >= 1:
        features.update({
            'kcc[-1]'  : kccs[i-1],
            'kcc[-1]t' : kcc_type(kccs[i-1]),
            'kcc[-1:0]': kccs[i-1] + kccs[i],
            'ns-1' : is_no_space(kccs[i-1])
        })
    else:
        features['BOS'] = True

    if i >= 2:
        features.update({
            'kcc[-2]'   : kccs[i-2],
            'kcc[-2]t'  : kcc_type(kccs[i-2]),
            'kcc[-2:-1]': kccs[i-2] + kccs[i-1],
            'kcc[-2:0]' : kccs[i-2] + kccs[i-1] + kccs[i],
        })
    if i >= 3:
        features.update({
            'kcc[-3]'   : kccs[i-3],
            'kcc[-3]t'  : kcc_type(kccs[i-3]),
            'kcc[-3:0]' : kccs[i-3] + kccs[i-2] + kccs[i-1] + kccs[i],
            'kcc[-3:-1]': kccs[i-3] + kccs[i-2] + kccs[i-1],
            'kcc[-3:-2]': kccs[i-3] + kccs[i-2],
        })

    if i < maxi-1:
        features.update({
            'kcc[+1]'  : kccs[i+1],
            'kcc[+1]t'  : kcc_type(kccs[i+1]),
            'kcc[+1:0]': kccs[i] + kccs[i+1],
            'ns+1' : is_no_space(kccs[i+1])

        })
    else:
        features['EOS'] = True

    if i < maxi-2:
        features.update({
            'kcc[+2]'   : kccs[i+2],
            'kcc[+2]t'   : kcc_type(kccs[i+2]),
            'kcc[+1:+2]': kccs[i+1] + kccs[i+2],
            'kcc[0:+2]' : kccs[i+0] + kccs[i+1] + kccs[i+2],
            'ns+2' : is_no_space(kccs[i+2])
        })
    if i < maxi-3:
        features.update({
            'kcc[+3]'   : kccs[i+3],
            'kcc[+3]t'   : kcc_type(kccs[i+3]),
            'kcc[+2:+3]': kccs[i+2] + kccs[i+3],
            'kcc[+1:+3]': kccs[i+1] + kccs[i+2] + kccs[i+3],
            'kcc[0:+3]' : kccs[i+0] + kccs[i+1] + kccs[i+2] + kccs[i+3],
        })

    return features

def generate_kccs_label_per_phrase(sentence):
    phrases = sentence.split()
    print("prep_kcc_labels -- number of phrases:", len(phrases))
    final_kccs = []
    for phrase in phrases:
        kccs = seg_kcc(phrase)
        labels = [1 if (i==0) else 0 for i, k in enumerate(kccs)]
        final_kccs.extend(list(zip(kccs,labels)))
    return final_kccs

def create_kcc_features(kccs):
    return [kcc_to_features(kccs, i) for i in range(len(kccs))]

# take label in second element from kcc with label
def create_labels_from_kccs(kccs_label):
    return [str(part[1]) for part in kccs_label]

# test
ts = "នៅ រសៀល ថ្ងៃ ទី ២២ ខែ កក្កដា ឆ្នាំ ២០១៩ ។"
kccs = seg_kcc(ts)
kccs_label = gen_kcc_with_label(ts) # only need for training
print("kcc with label:", kccs_label )
print("format features:", create_kcc_features(kccs))
print("create labels:", create_labels_from_kccs(kccs_label))
print("create labels:", create_kcc_features(seg_kcc("NS123")))

kcc with label: [('នៅ', 1), ('រ', 1), ('សៀ', 0), ('ល', 0), ('ថ្ងៃ', 1), ('ទី', 1), ('២២', 1), ('ខែ', 1), ('ក', 1), ('ក្ក', 0), ('ដា', 0), ('ឆ្នាំ', 1), ('២០១៩', 1), ('។', 1)]
format features: [{'kcc': 'នៅ', 't': 'K2', 'ns': False, 'BOS': True, 'kcc[+1]': 'រ', 'kcc[+1]t': 'C', 'kcc[+1:0]': 'នៅរ', 'ns+1': False, 'kcc[+2]': 'សៀ', 'kcc[+2]t': 'K2', 'kcc[+1:+2]': 'រសៀ', 'kcc[0:+2]': 'នៅរសៀ', 'ns+2': False, 'kcc[+3]': 'ល', 'kcc[+3]t': 'C', 'kcc[+2:+3]': 'សៀល', 'kcc[+1:+3]': 'រសៀល', 'kcc[0:+3]': 'នៅរសៀល'}, {'kcc': 'រ', 't': 'C', 'ns': False, 'kcc[-1]': 'នៅ', 'kcc[-1]t': 'K2', 'kcc[-1:0]': 'នៅរ', 'ns-1': False, 'kcc[+1]': 'សៀ', 'kcc[+1]t': 'K2', 'kcc[+1:0]': 'រសៀ', 'ns+1': False, 'kcc[+2]': 'ល', 'kcc[+2]t': 'C', 'kcc[+1:+2]': 'សៀល', 'kcc[0:+2]': 'រសៀល', 'ns+2': False, 'kcc[+3]': 'ថ្ងៃ', 'kcc[+3]t': 'K4', 'kcc[+2:+3]': 'លថ្ងៃ', 'kcc[+1:+3]': 'សៀលថ្ងៃ', 'kcc[0:+3]': 'រសៀលថ្ងៃ'}, {'kcc': 'សៀ', 't': 'K2', 'ns': False, 'kcc[-1]': 'រ', 'kcc[-1]t': 'C', 'kcc[-1:0]': 'រសៀ', 'ns-1': False, 'kcc[-2]': '

In [0]:
# load the model from disk
loaded_model = pickle.load(open(file, 'rb'))


In [0]:
# Create KCC using phrase, but pass in the whole sentence to CRF
def segment_kcc_phrase(sentence):
  complete = ""
  sentence = sentence.replace(u'\u200b','')
  for ph in sentence.split():
    kccs = seg_kcc(ph)

    features = create_kcc_features(kccs)
    prediction = loaded_model.predict([features])
    for i, p in enumerate(prediction[0]):
        if p == "1":
            complete += " " + kccs[i]
        else:
            complete += kccs[i]
    complete += " "
  complete=complete.strip()  
  return complete

#testing call  
t = "ចំណែកជើងទី២២២២ នឹងត្រូវធ្វើឡើងឯប្រទេសកាតា៕\nពួកគាត់តវ៉ាប្ដឹងទៅមេឃុំចៅវ៉ាយស្រុកជាច្រើនលើកដែរ ។"
t_correct = "ចំណែក ជើង ទី ២២២២ នឹង ត្រូវ ធ្វើឡើង ឯ ប្រទេស កាតា ៕\nពួកគាត់ ត វ៉ាប្ដឹង ទៅ មេឃុំ ចៅវ៉ាយ ស្រុក ជាច្រើន លើក ដែរ ។ "
skcc = seg_kcc(t)
print("len kcc:", len(skcc), skcc)
features = create_kcc_features(skcc)
features = [features]
print("features:", features)
print("\npredict seg:", loaded_model.predict(features))
print("\nseg:", segment_kcc_phrase(t))


len kcc: 46 ['ចំ', 'ណែ', 'ក', 'ជើ', 'ង', 'ទី', '២២២២', 'នឹ', 'ង', 'ត្រូ', 'វ', 'ធ្វើ', 'ឡើ', 'ង', 'ឯ', 'ប្រ', 'ទេ', 'ស', 'កា', 'តា', '៕', '\n', 'ពួ', 'ក', 'គា', 'ត់', 'ត', 'វ៉ា', 'ប្ដឹ', 'ង', 'ទៅ', 'មេ', 'ឃុំ', 'ចៅ', 'វ៉ា', 'យ', 'ស្រុ', 'ក', 'ជា', 'ច្រើ', 'ន', 'លើ', 'ក', 'ដែ', 'រ', '។']
features: [[{'kcc': 'ចំ', 't': 'K2', 'ns': False, 'BOS': True, 'kcc[+1]': 'ណែ', 'kcc[+1]t': 'K2', 'kcc[+1:0]': 'ចំណែ', 'ns+1': False, 'kcc[+2]': 'ក', 'kcc[+2]t': 'C', 'kcc[+1:+2]': 'ណែក', 'kcc[0:+2]': 'ចំណែក', 'ns+2': False, 'kcc[+3]': 'ជើ', 'kcc[+3]t': 'K2', 'kcc[+2:+3]': 'កជើ', 'kcc[+1:+3]': 'ណែកជើ', 'kcc[0:+3]': 'ចំណែកជើ'}, {'kcc': 'ណែ', 't': 'K2', 'ns': False, 'kcc[-1]': 'ចំ', 'kcc[-1]t': 'K2', 'kcc[-1:0]': 'ចំណែ', 'ns-1': False, 'kcc[+1]': 'ក', 'kcc[+1]t': 'C', 'kcc[+1:0]': 'ណែក', 'ns+1': False, 'kcc[+2]': 'ជើ', 'kcc[+2]t': 'K2', 'kcc[+1:+2]': 'កជើ', 'kcc[0:+2]': 'ណែកជើ', 'ns+2': False, 'kcc[+3]': 'ង', 'kcc[+3]t': 'C', 'kcc[+2:+3]': 'ជើង', 'kcc[+1:+3]': 'កជើង', 'kcc[0:+3]': 'ណែកជើង'}, {'kcc': 'ក', 

In [0]:
def crf_segment(t):
  t = cleanup_str(t)
  kccs = seg_kcc(t)
  features = create_kcc_features(kccs)
  features = [features]
  return segment_kcc_phrase(t)

def cleanup_str(str):
  #str = correct_str(str)
  # remove special characters
  str = str.replace(u"\u2028", "") # line separator
  str = str.replace(u"\u200a", "")  # hair space
  str = str.replace("<br>"," ") # changed to space
  return str.strip().replace('\n',' ').replace('  ',' ')

## Khmer Wikipedia model

### Download Khmer Wikipedia data

Wikipedia limit the number of download per day. Good to save the data.

In [0]:
from nlputils import split_wiki,get_wiki

In [0]:
#only need to do this once -- comment me if need to rerun
get_wiki(path,lang)

downloading...


unzipping...
extracting...


In [0]:
path.ls()

In [0]:
!head -n4 {path}/{name}

<doc id="1092" url="https://km.wikipedia.org/wiki?curid=1092" title="សេដ្ឋកិច្ច">
សេដ្ឋកិច្ច

យោងតាមវចនានុក្រមខ្មែររៀបរៀងដោយ សម្តេចសង្ឃរាជ ជួន ណាត (១៩៦២) ដែលខាងវិទ្យាស្ថានភាសាជាតិបានអំពាវនាវឲ្យប្រើប្រាស់ជាគោល​នោះ​ ​បាន​ពន្យល់​ពាក្យ​សេដ្ឋកិច្ចថា​បាន​មក​ពី​បន្សំ​រវាង​ពាក្យ សេដ្ឋ + កិច្ចការ ក្នុង​នោះ​ពាក្យ"សេដ្ឋ" មាន​ន័យ​ថា ប្រសើរ ឧត្តុង្គឧត្តម។​ ​រួម​សេចក្តី​មក សេដ្ឋកិច្ច​មាន​ន័យ​ថា​ជា​កិច្ចការ​ដ៏ឧត្តុង្គ​ឧត្តម​ថ្លៃថ្លា ឬ​ក៏​ជា​កិច្ចការ​ទាំង​ឡាយ​ដែលធ្វើ​ឲ្យ​ប្រទេស​ជាតិ​រីក​ចំរើន ។ សំណេរក្រោយ​មកមានការ​ប្រើ​ប្រាស់ពាក្យនេះក្នុងន័យប្រាស​ចាក​ពី​ន័យ​ដើម​របស់វា ពោលគឺនៅពេល​ដែលពាក្យនេះត្រូវបានយកទៅប្រើឲ្យសមន័យទៅនឹងពាក្យបារាំងឬអង់គ្លេសថា អេកូនូមី (Economy)។ លោក រ. គោវិទ (១៩៦៧) បានពន្យល់ពាក្យសេដ្ឋកិច្ច នេះថា ជាការសន្សំទុកឲ្យចំរើនឡើង ឬជាកិច្ចការរាជការដែលទាក់ទងនឹងពាណិជ្ជកម្ម និងឧស្សាហកម្មទូទៅ ។ ចំណែកលោក ហ៊ាន សាហ៊ីប និងហ៊ីង ថូរ៉ាក់ស៊ី (២០០៤) បាន បញ្ជាក់ពីការប្រើប្រាស់ពាក្យនេះសំដៅទៅលើអត្ថន័យ​បីជាសំខាន់ ១) សេដ្ឋកិច្ចគឺជាសំណុំនៃទំនាក់ទំនងផលិតកម្មសង្គម និង ជាខឿនទៃទំនាក់ទំនងនេះ ២) សេដ្ឋកិច្ចក្នុងន័យជាវិស័យ 

In [0]:
print(name)
!grep "doc id" /root/.fastai/data/kmwiki/kmwiki | wc #2536


kmwiki
   2536   10852  305220


In [0]:
!ls -alh /root/.fastai/data/kmwiki/
#!echo {name} #kmwiki
#!head /root/.fastai/data/kmwiki/kmwiki
#!cp -r /root/.fastai/data/kmwiki .
#!cp /root/.fastai/data/kmwiki/kmwiki kmwiki.txt
#!ls -alh #99M

total 322M
drwxr-xr-x 3 root root 4.0K Jan 30 00:21 .
drwxr-xr-x 3 root root 4.0K Jan 30 00:18 ..
-rw-r--r-- 1 root root 109M Jan 30 00:21 kmwiki
-rw-r--r-- 1 root root 191M Jan 30 00:20 kmwiki-latest-pages-articles.xml
-rw-r--r-- 1 root root  23M Jan 30 00:20 kmwiki-latest-pages-articles.xml.bz2
-rw-r--r-- 1 root root    0 Jan 30 00:20 log
drwxr-xr-x 3 root root 4.0K Jan 30 00:20 wikiextractor


### Save/Copy kmwiki to/from gdrive

In [0]:
# zip the file to save space
!cd /root/.fastai/data/kmwiki && zip kmwiki.zip kmwiki
!ls -alh  /root/.fastai/data/kmwiki/kmwiki.zip #99MB to 16MB

  adding: kmwiki (deflated 85%)
-rw-r--r-- 1 root root 17M Jan 30 00:29 /root/.fastai/data/kmwiki/kmwiki.zip


In [0]:
# copy to gdrive
!cp /root/.fastai/data/kmwiki/kmwiki.zip drive/'My Drive'/kmwiki_2020_01_29.zip
!ls  -alh drive/'My Drive'/kmwiki_*.zip

-rw------- 1 root root 16M Jan 13 19:46 'drive/My Drive/kmwiki_2020_01_13.zip'
-rw------- 1 root root 17M Jan 30 00:30 'drive/My Drive/kmwiki_2020_01_29.zip'
-rw------- 1 root root 18M Jan 14 22:23 'drive/My Drive/kmwiki_seg.zip'


In [0]:
#copy from gdrive -- you can get this file from github
!mkdir kmwiki
!cp drive/'My Drive'/kmwiki_2020_01_29.zip kmwiki
!cd kmwiki && unzip kmwiki_2020_01_29.zip

Archive:  kmwiki_2020_01_29.zip
  inflating: kmwiki                  


In [0]:
!ls -alh kmwiki/kmwiki 

-rw-r--r-- 1 root root 109M Jan 30 00:21 kmwiki/kmwiki


### Split data
This function splits the single wikipedia file into a separate file per article. This is often easier to work with.

In [0]:
from pathlib import Path
lpath = Path('./kmwiki')
lpath

PosixPath('kmwiki')

In [0]:
from nlputils import split_wiki,get_wiki
# split the data file into multiple text file
dest = split_wiki(lpath,lang)

0
100000
200000


In [0]:
dest.ls()[:5] ### kmwiki/docs/*.txt
#len(dest.ls()) #2536
#dest


[PosixPath('kmwiki/docs/មីសឺន.txt'),
 PosixPath('kmwiki/docs/ប្រវត្តិវិទ្យុ.txt'),
 PosixPath('kmwiki/docs/ព្រះសហគមន៏កាតូលិកភូមិភាគភ្នំពេញ.txt'),
 PosixPath('kmwiki/docs/ខេត្តឃ្លាំង.txt'),
 PosixPath('kmwiki/docs/ភាគ១៥.txt')]

In [0]:
!head -3 kmwiki/docs/អង្គការតម្លាភាពកម្ពុជា.txt

អង្គការតម្លាភាពកម្ពុជា

អង្គការបានជ្រើសយកអភិបាលកិច្ចល្អគឺជាលក្ខខណ្ឌមុនដ៏សំខាន់បំផុតក្នុងការសម្រេចបាននូវការអភិវឌ្ឍសេដ្ឋកិច្ចប្រកបដោយនិរន្តរភាព។ ហើយក៏តម្រូវឱ្យមានការចូលរួមយ៉ាងសកម្មនិង​ការប្តេជ្ញាចិត្ត​របស់​សាធារណជនគ្រប់មជ្ឈដ្ឋានក្នុងសង្គមនិងប្រជាពលរដ្ឋកម្ពុជាមានសិទ្ធិទទួលបានប្រព័ន្ធអភិបាលកិច្ចសមត្ថកិច្ចថាគឺជាអំពើពុករលួយដោយឥតគិតថ្លៃគណនេយ្យភាពនិងតម្លាភាព។ 


### Segment the text into words using space

In [0]:
data_dir = dest.absolute() #/content/kmwiki/docs
print(data_dir)
!mkdir /content/kmwiki_data

/content/kmwiki/docs


In [0]:
def crf_segment(t):
  t = cleanup_str(t)
  kccs = seg_kcc(t)
  features = create_kcc_features(kccs)
  features = [features]
  return segment_kcc_phrase(t)

def cleanup_str(str):
  #str = correct_str(str)
  # remove special characters
  str = str.replace(u"\u2028", "") # line separator
  str = str.replace(u"\u200a", "")  # hair space
  str = str.replace("<br>"," ") # changed to space
  return str.strip().replace('\n',' ').replace('  ',' ')

In [0]:
# this takes over 10 mins

# foreach file, run through crf and write to new folder data/filename.txt
import glob
import re

data_dir = str(dest.absolute())
path = data_dir + '/*.txt'   
files=glob.glob(path)

print("Number of file to convert:", len(files))

# global variables that use through out
seg_text = []
orig_text = []
# unique id of the article that can be matched to docId in meta.txt
doc_ids = []
i=0
for file in files:
  f=open(file, 'r')
  lines = f.readlines()
  f.close()
  orig_text.append(lines)
  
  # limit to 9K to avoid memory issue
  #if len(orig_text) >= 10: break #kcc=9000, char:5000, crash char on 7000

  f_text = ""
  for t in lines:    
    if len(t.strip()) > 0: f_text = f_text + "\n" + crf_segment(t)
  foutname = "/content/kmwiki_data/" + os.path.basename(file)
  fout = open(foutname, 'w')
  fout.write(f_text)
  fout.close()
  #print("write to file:", foutname)
  if i % 100 == 0: print("convert file:", i)
  i = i +1
print("number of file:", len(files))
#print("len of seg_text:", len(seg_text), ' sample data[0][0]:', seg_text[0][0])
print("len of orig_text:", len(orig_text), ' sample data[0][0]:', orig_text[0][0])

Number of file to convert: 2536
convert file: 0
convert file: 100
convert file: 200
convert file: 300
convert file: 400
convert file: 500
convert file: 600
convert file: 700
convert file: 800
convert file: 900
convert file: 1000
convert file: 1100
convert file: 1200
convert file: 1300
convert file: 1400
convert file: 1500
convert file: 1600
convert file: 1700
convert file: 1800
convert file: 1900
convert file: 2000
convert file: 2100
convert file: 2200
convert file: 2300
convert file: 2400
convert file: 2500
number of file: 2536
len of orig_text: 2536  sample data[0][0]: មីសឺន



In [0]:
!head -5 /content/kmwiki_data/*Bible*.txt
!ls /content/kmwiki_data/  | wc


៦.  គម្គីរ ព្រះបរិសុទ្ធ  (Holy  Bible)
កាល ដើម ដំបូង ឡើយ  ព្រះបាន បង្កើត ផ្ទៃ មេឃ នឹង ផែនដី ។  "In  the  beginning  God  created  the  heavens  and  the  earth."
ឯ ផែនដី បាន ខូច  ហើយ នៅ ទទេ  មាន សុទ្ធតែ ងងឹត នៅ គ្រប លើ ជំរៅ ទឹក  ហើយ ព្រះវិញ្ញាណ នៃ  ព្រះក៏រេរា នៅ ពីលើ ទឹក  "Now  the  earth  was  formless  and  empty,  darkness  was  over  the  surface  of  the  deep,  and  the  spirit  of  God  was  hovering  over  the  waters."  នោះ ព្រះទ្រង់ មាន បន្ទូល ថា  ចូរ ឲ្យ មាន ពន្លឺ ឡើង  ដូច្នេះ ពន្លឺ ក៏ មាន ឡើង  "and  God  said  “  Let  there  be  light,”  and  there  was  light"  ព្រះទ្រង់ ឃើញ ពន្លឺ នោះ  ក៏ យល់ ថា ជា ល្អ ហើយ  រួច ទ្រង់ ញែក ពន្លឺ ពី ងងឹត ចេញ  "God  saw  that  the  light  was  good,  and  he  separated  the  light  from  the  darkness".  ទ្រង់ ហៅ ពន្លឺ ថា ជា ថ្ងៃ  ហើយ ហៅ ងងឹត ថា  ជា យប់  នោះ ក៏ មាន ល្ងាច មាន ព្រឹក ឡើង  ជា ថ្ងៃទី ១ ។  "God  called  the  light  “day,”  and  he  called  the  darkness  “night.”  And  there  evening,  and  there  was  morning__the  first  day."
បន

### Copy semented zip file to/from gdrive

In [0]:
# Save semented data into g drive  -- will contain /cotnet/kmwiki_data folder
seg_zip = 'kmwiki_seg_2020_01_29.zip'
!cd /content && zip {seg_zip} /content/kmwiki_data/*
!ls -al /content/*.zip

!cp /content/kmwiki_seg.zip drive/'My Drive'/{seg_zip}
!ls -alh drive/'My Drive'/kmwiki_seg*.zip

In [0]:
!ls -alh /content/kmwiki_seg.zip drive/'My Drive'/kmwiki_seg_1_29_2020.zip

In [0]:
# copy seg_file from g drive
!mkdir /content/kmwiki_data
!cp drive/'My Drive'/{seg_zip} /content

# unzip (zip file already have /content/kmwiki_data structure)
!cd / && unzip /content/{seg_zip} | tail -5


mkdir: cannot create directory ‘/content/kmwiki_data’: File exists
  inflating: content/kmwiki_data/Vibhaṅga.txt  
  inflating: content/kmwiki_data/កង់(wheel).txt  
  inflating: content/kmwiki_data/William Thomson, 1st Baron Kelvin.txt  
  inflating: content/kmwiki_data/World Cup.txt  
  inflating: content/kmwiki_data/វើលវ៉ាយវែប​ (World Wide Web).txt  


In [0]:
tempath = Path("kmwiki_data")
print(tempath.absolute())
len(tempath.ls()) #number of files 2538

/content/kmwiki_data


2538

In [0]:
!head -2 /content/kmwiki_data/វើលវ៉ាយវែប​*.txt  


វើល វ៉ាយ វែប  (World  Wide  Web)


In [0]:
sizes = ["100","500","1000","5000","10000"]
# Select available size (0-4) that you want to use
docsize = sizes[2]
data_dir = "kh_data_" + docsize
file_name = data_dir + ".zip" # need old file format with space

In [0]:
# Download the file from `url` and save it
import urllib.request
base_url = "https://github.com/phylypo/segmentation-crf-khmer/raw/master/data/"
url = base_url + file_name
print("Downloading from:", url)
urllib.request.urlretrieve(url, file_name)

('kh_data_1000.zip', <http.client.HTTPMessage at 0x7fb9531fb320>)

In [0]:
print(data_dir)

kh_data_1000


In [0]:
# remove previous existing directory for rerun
!rm -rf {data_dir}
print("- Unzipping the file and show last few extracted files:")
!unzip {file_name} | tail -10

print("- Count the number of files:")
!ls -al {data_dir}/*_seg*.txt | wc -l

- Unzipping the file and show last few extracted files:
  inflating: kh_data_1000/313546_seg.txt  
  inflating: kh_data_1000/313565_orig.txt  
  inflating: kh_data_1000/313565_seg.txt  
  inflating: kh_data_1000/313575_orig.txt  
  inflating: kh_data_1000/313575_seg.txt  
  inflating: kh_data_1000/313576_orig.txt  
  inflating: kh_data_1000/313576_seg.txt  
  inflating: kh_data_1000/313577_orig.txt  
  inflating: kh_data_1000/313577_seg.txt  
  inflating: kh_data_1000/meta.txt   
- Count the number of files:
1000


In [0]:
!head -1 kh_data_1000/313546_seg.txt
!head -1 kh_data_1000/313546_orig.txt

 លោកស្រី សុផុន ប្រធាន ធម្មការ ស្រុក ស្រី ស្នំ ឃុបឃិត ជាមួយ  មេឃុំ  មេភូមិ  លក់ ដី សហគម ន៏  ជិង ២០០ ហិកតា
លោកស្រី​សុ​ផុន ប្រធាន​ធម្មការ​ស្រុក​ស្រីស្នំ ឃុបឃិត ជាមួយ មេឃុំ មេភូមិ លក់​ដី​សហគម​ន៏ ជិ​ង ២០០ ហិកតា


In [0]:
!cp -rf kh_data_1000/*_seg.txt  /content/kmwiki_data/

tempath = Path("kmwiki_data")
len(tempath.ls()) #number of files 7536

3538

### Create pretrained model

In [0]:
#should set to kmwiki_data --segemented
segpath = Path('kmwiki_data')
print(segpath)

kmwiki_data


In [0]:
data = (TextList.from_folder(segpath) # change from dest to segpath
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(53656, 3183)

In [0]:
print(len(data.vocab.itos)) #53k , no crf=50k, 1k CRF= 53656, 5K CRF=59408
print(data.vocab.itos[10:20])

53656
['មាន', 'ជា', 'បាន', '\n ', 'ដែល', 'ថា', 'មិន', 'នោះ', 'ក្នុង', 'នេះ']


In [0]:
data.show_batch()

idx,text
0,កម្មវិធី វិនិយោគ បី ឆ្នាំ រំកិល ប្រចាំ ឆ្នាំ \n - ក្របខ័ណ្ឌ ចំណាយ រយៈពេល មធ្យម ដែល ត្រូវ ធ្វើ បច្ចុប្បន្នភាព ជារៀងរាល់ ឆ្នាំ \n - គម្រោង ថវិកា ប្រចាំ ឆ្នាំ \n - ការបង្កើត ការកែសម្រួល ឬ ការរំសាយ ទី ចាត់ការ ឬ ការិយាល័យ នានា \n - ការរៀបចំ តួនាទី ភារកិច្ច និង លក្ខខណ្ឌ ការងារ របស់ បុគ្គលិក \n - ការតែងតាំង ការដំឡើង ថ្នាក់ ឋានន្ដរសក្ដិ និង ការបញ្ឈប់ បុគ្គលិក \n - ការកំណត់ xxunk នៃ ការជ្រើសរើស ការតែងតាំង ការកំណត់ ប្រាក់
1,បាន សាង ធ្វើ ប្រាសាទ និង បូជនីយដ្ឋាន អំពី ថ្ម ដោយ មាន ទាំង ជីក បារាយណ៍ សង់ ស្ពាន និង ថ្នល់ ផង ។ \n តែ បើ តាម “ ប្រជុំ រឿង ព្រេង ខ្មែរ ភាគ ទី ៥ ” របស់ ពុទ្ធសាសន បណ្ឌិត្យ វិញ ព្រះពិស្ណុការ មិន មែន ចុះ មកពី ឋានសួគ៌ តាម បញ្ជា ព្រះឥន្ទ្រាធិ រាជ ទេ ។ xxunk កូនប្រុស របស់ ទេ xxunk ទិព្ធ សុតា ច័ន្ទ និង ចិក លឹម សេង ដែល រស់ នៅ ក្រុង សៀងហៃ នៃ ប្រទេស ចិន ។
2,"\n "" ម្នាល ឧបាសក ! កុំ ខ្វល់ នឹក ដល់ កូនប្រុស របស់ ញោម ខ្លាំង ពេក ចូរ ធ្វើ ចិត្ត ឲ្យ សប្បាយ ចុះ គេ មាន សេចក្តីសុខ ហើយ ចូរ ខ្វល់ រឿង ខ្លួនឯង ឲ្យ មែនទែន ពីព្រោះ ញោម ចាស់ជរា ណាស់ ហើយ ញោម ប្រៀប ដូចជា ដើមឈើ ដែល ដុះ នៅក្បែរ ច្រាំង ស្ទឹង គឺ មច្ចុរាជ ទោះបីជា ញោម នឹក ដល់ កូនប្រុស របស់ ញោម ប៉ុន ណា គេ ក៏ ជួយ អ្វី ញោម មិន បាន ដែរ ខ្លួន ញោម ប៉ុណ្ណោះ ដែល អាច ជួយ"
3,កូន អញ ស៊ី xxunk ជើង ដែល អញ បាន ឃើញ និង ជើង ខ្លា នេះ មាន ទំហំ ប៉ុន គ្នា ហើយ មិន ឃើញ មាន xxunk ណា ធំ ដូច ខ្លា នេះ ផង មាន តែ ខ្លា នេះ ហើយ ជា មេហ្វូង ធំ ជាង គេ បានជា ហ៊ាន ចូលទៅ ខាំ កូន អញ ស៊ី ទាំង ថ្ងៃ ដូច្នេះ ។ តា ពស់ ព្រៃ គិត ថា បើ អញ សម្លាប់ ខ្លា នេះ ដោយ កំលាំង បាយ មុខ ជាមិន ឈ្នះ វា ទេ គួរគប្បី អញ ប្រើ
4,ពួក សត្វ ទាំងអស់ ក៏ សន្យា នឹង តោ ថា នឹង គោរព តាម សន្យា ។ ដូច្នេះហើយ ជារៀងរាល់ ថ្ងៃ ពួកគេ បញ្ជូន សត្វ មួយ ក្បាល ទៅ តោ ជា ចំណី រួចហើយ រកស៊ី ក្នុង ព្រៃ ដោយ គ្មាន ខ្លាច កោត ក្រែង តោ អ្វី ទៀត ដែរ ។ \n ថ្ងៃ មួយ ជា ថ្ងៃ ដែល ត្រូវ ដល់ វេណ ទន្សាយ ។ ដោយសារតែ ការបង្ខំ ពី សត្វ ផ្សេង ទៀត ទន្សាយ ក៏ ធ្វើដំណើរ ទៅ ជួប នឹង តោ យ៉ាង យឺត ៗ ទាំង ចិត្ត មិន


### Save/Load databunch from gdrive

In [0]:
## save databunch to drive
!cp kmwiki_data/km_databunch drive/'My Drive'/khmer_ulmfit/km_databunch_1k
!ls -alh drive/'My Drive'/khmer_ulmfit/km_*

## copy back to folder
#!mkdir kmwiki_data
#!cp  drive/'My Drive'/km_databunch kmwiki_data
#!ls -alh kmwiki_data

## save to folder
#!cp  drive/'My Drive'/km_databunch drive/'My Drive'/khmer_ulmfit/km_databunch

-rw------- 1 root root  66M Jan 13 21:38 'drive/My Drive/khmer_ulmfit/km_databunch'
-rw------- 1 root root  76M Feb  1 23:47 'drive/My Drive/khmer_ulmfit/km_databunch_1k'
-rw------- 1 root root  88M Jan 31 22:05 'drive/My Drive/khmer_ulmfit/km_databunch_5k'
-rw------- 1 root root  73M Jan 31 03:22 'drive/My Drive/khmer_ulmfit/km_databunch_bwd'
-rw------- 1 root root 3.7M Jan 15 00:43 'drive/My Drive/khmer_ulmfit/km_textlist_class'


In [0]:
# load databunch from g drive
!mkdir kmwiki_data
!cp drive/'My Drive'/khmer_ulmfit/km_databunch_1k kmwiki_data/km_databunch

mkdir: cannot create directory ‘kmwiki_data’: File exists


In [0]:
!ls kmwiki_data/*databunch
segpath = Path('kmwiki_data')
print(segpath)
len(segpath.ls()) #number of files 7537 -- no seg file when reload in this section

kmwiki_data/km_databunch
kmwiki_data


3538

In [0]:
bs=64 #64
lang="km"

In [0]:
# change segpath to use one in drive
segpath = Path('kmwiki_data')
data = load_data(segpath, f'{lang}_databunch', bs=bs)
print(segpath)

kmwiki_data


In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size
lr
#del data
#torch.cuda.empty_cache()

0.013333333333333332

In [0]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7)) # should be 10 cycle

# with crf data
#epoch	train_loss	valid_loss	accuracy	time
#0	4.763054	4.983258	0.215832	04:34
#9	3.236668	3.992879	0.326933	04:37
#without crf data
#0	4.757779	5.117013	0.199202	04:33
#8	3.704572	3.885062	0.348298	04:37
#ram usage 3GB bs=64
#0	4.792127	4.929679	0.215631	04:52
#9	3.371586	3.715175	0.364118	04:49
#10k CRF data
#0	4.586102	4.879279	0.228851	06:34
#9	3.388669	3.767544	0.352145	06:34
#10k crf data diff run
#0	4.526210	4.900151	0.222933	08:26
#9	3.292509	3.770821	0.353640	08:27
#1k crf data
#0	4.668200	4.789621	0.241485	05:35
#9	3.262789	3.734351	0.368645	05:33

epoch,train_loss,valid_loss,accuracy,time
0,4.668200,4.789621,0.241485,05:35
1,4.394073,4.653580,0.249886,05:34
2,4.354604,4.591842,0.256225,05:34
3,4.279814,4.425426,0.274856,05:33
4,4.162161,4.260066,0.293360,05:32
5,4.022367,4.110199,0.311260,05:32
6,3.729630,3.941348,0.336584,05:33
7,3.518007,3.822778,0.351697,05:33
8,3.290968,3.745364,0.365342,05:33
9,3.262789,3.734351,0.368645,05:33


In [0]:
#learn.fit_one_cycle(5, lr, moms=(0.8,0.7)) # should not run this
# 5k crf -- start off bad
#0	3.708809	4.128892	0.303718	08:24
#4	3.202902	3.762015	0.351793	08:23
#1k crf
#0	3.673259	4.046010	0.326937	05:33
#4	3.470854	3.704953	0.369526	05:33

epoch,train_loss,valid_loss,accuracy,time
0,3.673259,4.046010,0.326937,05:33
1,4.054169,4.196497,0.305048,05:34
2,3.898169,4.007215,0.326522,05:33
3,3.469066,3.770610,0.359164,05:34
4,3.470854,3.704953,0.369526,05:33


In [0]:
path = Path('kmwiki_data')
print(path) # should be set to current /content
# save model to google drive
#!mkdir drive/'My Drive'/khmer_ulmfit
#path = Path("drive/'My Drive'/khmer_ulmfit")
#path.mkdir() #exist_ok=True)

kmwiki_data


Save the pretrained model and vocab:

In [0]:
#!mkdir drive/'My Drive'/khmer_ulmfit/models
mdl_path = path/'models'
print("lm_fns:",mdl_path/lm_fns[0])
print("lm_fns vocab:",lm_fns[1])
#mdl_path.mkdir(exist_ok=True)
#learn.to_fp32().save("km_wt", with_opt=False) # save weight of pretrain

learn.to_fp32().save("km_wt", with_opt=False) # this save to: kmwiki_data/models/km_wt.pth
#learn.data.vocab.save((lm_fns[1] + '.pkl'))
learn.data.vocab.to_fp32().save((lm_fns[1] + '.pkl')) #force to 32 bit ??


lm_fns: kmwiki_data/models/km_wt
lm_fns vocab: km_wt_vocab


In [0]:
!ls  -alh kmwiki_data/models/
!date
#-rw-r--r-- 1 root root  83793398 Jan 16 00:24 km_wt.pth
#-rw-r--r-- 1 root root 418959901 Jan 16 00:20 km_wt_test.pth

total 783M
drwxr-xr-x 2 root root 4.0K Feb  2 01:12 .
drwxr-xr-x 4 root root 236K Feb  2 01:12 ..
-rw------- 1 root root 207M Feb  1 23:34 kmclas.pth
-rw------- 1 root root 126M Feb  1 23:34 kmfine_tuned_enc.pth
-rw------- 1 root root 290M Feb  1 23:34 kmfine_tuned.pth
-rw------- 1 root root 160M Feb  2 01:12 km_wt.pth
-rw------- 1 root root 1.4M Feb  1 23:34 km_wt_vocab.pkl
Sun Feb  2 01:13:43 UTC 2020


In [0]:
!mkdir kmwiki_data/models1k
!cp kmwiki_data/models/km_wt.pth kmwiki_data/models1k
!cp km_wt_vocab.pkl kmwiki_data/models1k
!ls -alh kmwiki_data/models1k


total 162M
drwxr-xr-x 2 root root 4.0K Feb  2 01:12 .
drwxr-xr-x 4 root root 236K Feb  2 01:12 ..
-rw------- 1 root root 160M Feb  2 01:12 km_wt.pth
-rw-r--r-- 1 root root 1.4M Feb  2 01:12 km_wt_vocab.pkl


In [0]:
!ls kmwiki_data/models1k
#km_wt.pth  km_wt_vocab.pkl

km_wt.pth  km_wt_vocab.pkl


In [0]:
#backup to google drive
!mkdir drive/'My Drive'/khmer_ulmfit/models1k
!cp kmwiki_data/models1k/* drive/'My Drive'/khmer_ulmfit/models1k
!ls -alh drive/'My Drive'/khmer_ulmfit/models1k

mkdir: cannot create directory ‘drive/My Drive/khmer_ulmfit/models1k’: File exists
total 783M
-rw------- 1 root root 207M Jan 21 01:25 kmclas.pth
-rw------- 1 root root 126M Jan 21 01:25 kmfine_tuned_enc.pth
-rw------- 1 root root 290M Jan 21 01:25 kmfine_tuned.pth
-rw------- 1 root root 160M Feb  2 01:13 km_wt.pth
-rw------- 1 root root 1.4M Feb  2 01:13 km_wt_vocab.pkl


### Copy model file from gdrive

In [0]:
# copy model files from g drive
mtype = '1k' # 1k or 5k
!mkdir -p kmwiki_data/models
!cp drive/'My Drive'/khmer_ulmfit/models{mtype}/* kmwiki_data/models/

In [0]:
# copy databunch
!cp  drive/'My Drive'/khmer_ulmfit/km_databunch_{mtype} kmwiki_data
!mv kmwiki_data/km_databunch_{mtype} kmwiki_data/km_databunch
!ls -alh kmwiki_data/km*

-rw------- 1 root root 76M Feb  9 03:08 kmwiki_data/km_databunch


### Copy file from gdrive share url

In [0]:
#taken from this StackOverflow answer: https://stackoverflow.com/a/39225039 (https://github.com/nsadawi/Download-Large-File-From-Google-Drive-Using-Python)
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
model_dir = 'kmwiki_data/models/'
!mkdir -p {model_dir}

In [9]:
# Download the file from `url` and save it
file_ids = ['1-1sQxNhWCdsicuk97ikMorb4fzpfo4Ib','1-1Mf9rFrN-d_FlkiQ3J4K1hYYTgU0QQ7','1-9ZKLkhv57IQ6qPKsSeDTdF225QQ8x7W']
destinations = ['km_wt_vocab.pkl','km_wt.pth','kmclas.pth']

for i in [0,1,2]:
  print("file:", destinations[i])
  download_file_from_google_drive(file_id, model_dir + destination)


file: km_wt_vocab.pkl
file: km_wt.pth
file: kmclas.pth


In [23]:
# download databunch from github
filename = 'km_databunch'
import urllib.request
url = 'https://github.com/phylypo/khmer-language-model-ulmfit/blob/master/data/km_databunch_1k?raw=true'
print("url", url)
urllib.request.urlretrieve(url, "kmwiki_data/" + filename)

url https://github.com/phylypo/khmer-language-model-ulmfit/blob/master/data/km_databunch_1k?raw=true


('kmwiki_data/km_databunch', <http.client.HTTPMessage at 0x7fe19a5cd860>)

In [24]:
!ls -alh kmwiki_data
!ls -alh kmwiki_data/models

total 76M
drwxr-xr-x 3 root root 4.0K Feb 16 23:31 .
drwxr-xr-x 1 root root 4.0K Feb 16 23:22 ..
-rw-r--r-- 1 root root  76M Feb 16 23:31 km_databunch
drwxr-xr-x 2 root root 4.0K Feb 16 23:21 models
total 368M
drwxr-xr-x 2 root root 4.0K Feb 16 23:21 .
drwxr-xr-x 3 root root 4.0K Feb 16 23:31 ..
-rw-r--r-- 1 root root 207M Feb 16 23:21 kmclas.pth
-rw-r--r-- 1 root root 160M Feb 16 23:21 km_wt.pth
-rw-r--r-- 1 root root 1.4M Feb 16 23:21 km_wt_vocab.pkl


### Load model from pkl file

In [25]:
segpath = Path('kmwiki_data')
print(segpath)
!ls -alh kmwiki_data/models

kmwiki_data
total 368M
drwxr-xr-x 2 root root 4.0K Feb 16 23:21 .
drwxr-xr-x 3 root root 4.0K Feb 16 23:31 ..
-rw-r--r-- 1 root root 207M Feb 16 23:21 kmclas.pth
-rw-r--r-- 1 root root 160M Feb 16 23:21 km_wt.pth
-rw-r--r-- 1 root root 1.4M Feb 16 23:21 km_wt_vocab.pkl


In [0]:
lang='km'
bs=40

In [0]:
# require setup path to kmwiki_data/km_databunch
data = load_data(segpath, f'{lang}_databunch', bs=bs)
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [28]:
path = 'kmwiki_data/models'
itos_fname = path + '/km_wt_vocab.pkl'
wgts_fname = path + '/km_wt.pth'
learn.to_fp16().load_pretrained(wgts_fname, itos_fname, True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (3183 items)
x: LMTextList
xxbos 
  គណៈ អភិបាល ខេត្ត 
  ខេត្ត នីមួយ ៗ ត្រូវ មាន គណៈ អភិបាល ខេត្ត មួយ ដែល រួមមាន អភិបាល ខេត្ត និង អភិបាល រង ខេត្ត មួយចំនួន ដែល ត្រូវបាន តែងតាំង ឡើង ស្រប តាម ល័ក្ខខ័ណ្ឌ នៃ ច្បាប់ ស្ដីពី ការគ្រប់គ្រង រដ្ឋបាល រាជធានី ខេត្ដ ក្រុង ស្រុក ខណ្ឌ ។ អភិបាល ខេត្ត អភិបាល រង ខេត្ត មិន មែន ជា សមាជិក ក្រុមប្រឹក្សា ខេត្ត ឡើយ ។ 
  គណៈ អភិបាល ខេត្ត មាន តួនាទី និង ភារកិច្ច ចម្បង ៗ ដូច ខាងក្រោម ៖ 
  - ការពិនិត្យ មុខងារ ភារកិច្ច និង ធនធាន ទាំងអស់ ដែល បាន ប្រគល់ ឬ បាន ផ្ទេរ មក ឱ្យ ក្រុមប្រឹក្សា ខេត្ត និង អំពី របៀបរបប បែបបទ នីតិវិធី នៃ ការអនុវត្ដ មុខងារ ភារកិច្ច និង ការចាត់ចែង ធនធាន ទាំងនោះ 
  - ការរៀបចំ យុទ្ធសាស្ដ្រ និង ការរៀបចំ រចនាសម្ព័ន្ធ ប្រព័ន្ធ និង ធនធាន ដើម្បី ទទួលយក មុខងារ ភារកិច្ច និង ធនធាន ដែល បាន ប្រគល់ ឬ បាន ផ្ទេរ ស្រប តាម ច្បាប់ ស្ដីពី ការគ្រប់គ្រង រដ្ឋបាល រាជធានី ខេត្ដ ក្រុង ស្រុក ខណ្ឌ ។ 
  - ផែនការអភិវឌ្ឍន៍ រយៈពេល ប្រាំ ឆ្នាំ និង កម្មវិធី វិនិយោគ បី ឆ្នាំ រំកិល ប្រចាំ ឆ្នាំ 
  - ក្របខ័ណ្ឌ ចំណាយ រយៈពេល មធ្យម 

In [0]:
TEXT = "ប្រាសាទ ភ្នំ"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#ប្រាសាទ ភ្នំ ប្រុស ជា ភ្នំ ធំ ជាង គេ ក្នុង ប្រទេស កម្ពុជា 
#ប្រាសាទ ភ្នំ ដងរែក ស្ថិត នៅក្នុង ភូមិ ក្រាំង ធំ ឃុំ ប្រចាំ ក្នុង ខេត្ត សៀមរាប 

ប្រាសាទ ភ្នំ ស្រី វិបុល រូប តួអង្គ តំណាង ឱ្យ ស្រី គ្រប់ លក្ខណ៍ ដែល មាន លក្ខណៈ ស្រដៀង និង ទម្រង់ សិល្បៈ មួយ ដែល មាន តម្លៃ
ប្រាសាទ ភ្នំ ជី សូរ ស្ថិត នៅក្នុង ភូមិ ព្រៃ ល្វា ឃុំ គោក ចក ស្រុក សំរោងទង ខេត្ត កណ្តាល ខេត្ត កំពង់ចាម ។ 
  ហ៊ុន សែន
ប្រាសាទ ភ្នំ ថ្ម បាយ ក្រៀម សម្រាប់ ជា ទី ជោគជ័យ ដើម្បី អារម្មណ៍ ១ 
  ជាទោស ដ៏ ខ្ពង់ខ្ពស់ នៃ សិល្បៈ គឺជា សិល្បៈ ដែល មាន
ប្រាសាទ ភ្នំ when អវិជ្ជា ដែល មាន ដំបូល ព័ទ្ធ ជុំវិញ ផ្ទះ មាន ភាព ដូចគ្នា មាន បន្ទប់ សម្រាប់ បើក ភ្នែក 
  ប្រភេទ នៃ ទ្វារ
ប្រាសាទ ភ្នំ ធំ ក្នុង ព្រះរាជាណាចក្រ កម្ពុជា ស្ថិត នៅ ភ្នំពេញ និង រាជធានី ភ្នំពេញ ។ 
  លោក បាន បញ្ជាក់ ថា ដូចជា អាច ធ្វើឲ្យ យើង


In [0]:
print(len(data.vocab.itos)) #53920, 51552 without crf, 5kcrf=59408, 1k=53656
print(data.vocab.itos[10:30])

53656
['មាន', 'ជា', 'បាន', '\n ', 'ដែល', 'ថា', 'មិន', 'នោះ', 'ក្នុង', 'នេះ', 'ហើយ', 'និង', 'ភិក្ខុ', 'ដោយ', 'របស់', 'ក៏', 'នូវ', 'ទៅ', 'ទាំងឡាយ', 'នៅ']


In [0]:
path = 'kmwiki_data/models'
print(path)
print(lm_fns[0])
print(lm_fns[1])
!ls -alh path

kmwiki_data/models
km_wt
km_wt_vocab
total 8.0K
drwxr-xr-x 2 root root 4.0K Feb  9 03:15 .
drwxr-xr-x 1 root root 4.0K Feb  9 03:17 ..


In [0]:
# test save vocab in 32 bit
print("lm_fns:",lm_fns[0])
print("lm_fns vocab:",lm_fns[1])
#mdl_path.mkdir(exist_ok=True)

learn.to_fp32().save("km_wt", with_opt=False) # this save to: kmwiki_data/models/km_wt.pth
#learn.data.vocab.save((lm_fns[1] + '.pkl'))
learn.to_fp32().data.vocab.save((lm_fns[1] + '.pkl')) #force to 32 bit ??

lm_fns: km_wt
lm_fns vocab: km_wt_vocab


In [0]:
!date
!cp km_wt_vocab.pkl kmwiki_data/models
!ls -alh kmwiki_data/models
#!md5sum kmwiki_data/models/km_wt_vocab.pkl
#!md5sum kmwiki_data/km_wt_vocab.pkl


Sun Feb  9 03:58:02 UTC 2020
total 826M
drwxr-xr-x 2 root root 4.0K Feb  9 03:58 .
drwxr-xr-x 4 root root 4.0K Feb  9 03:26 ..
-rw------- 1 root root 207M Feb  9 03:08 kmclas.pth
-rw------- 1 root root 126M Feb  9 03:08 kmfine_tuned_enc.pth
-rw------- 1 root root 290M Feb  9 03:08 kmfine_tuned.pth
-rw------- 1 root root 203M Feb  9 03:58 km_wt.pth
-rw------- 1 root root 1.4M Feb  9 03:58 km_wt_vocab.pkl


In [0]:
drv_model = "drive/'My Drive'/khmer_ulmfit/models_1k_32"
!mkdir {drv_model}
!cp kmwiki_data/models/km_wt_vocab.pkl {drv_model}/
!cp kmwiki_data/models/km_wt.pth {drv_model}/
!ls -alh {drv_model}

mkdir: cannot create directory ‘drive/My Drive/khmer_ulmfit/models_1k_32’: File exists
total 204M
-rw------- 1 root root 203M Feb  9 03:58 km_wt.pth
-rw------- 1 root root 1.4M Feb  9 03:58 km_wt_vocab.pkl


In [0]:
!ls -alh drive/'My Drive'/khmer_ulmfit/models_1k_32/


total 204M
-rw------- 1 root root 203M Feb  9 03:58 km_wt.pth
-rw------- 1 root root 1.4M Feb  9 03:58 km_wt_vocab.pkl


## Testing the model

In [0]:
TEXT = "ព្រះឣង្គ"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#ព្រះឣង្គ បាន បង្កើត ឆ្នេរសមុទ្រ ថ្មី មួយ ដែល មាន ទីតាំង នៅ ក្រុង រណប ខេត្ត បន្ទាយមានជ័យ និង ក្រុង ព្រះសីហនុ ។ 
#ព្រះឣង្គ ទ្រង់ ប្រទាន ព័ត៌មាន មិន ពិត មួយ ស្ថានភាព នៃ ព័ត៌មាន ក្លែងក្លាយ ផ្ទាល់ ព្រមទាំង ព័ត៌មាន ក្លែងក្លាយ និង រូបភាព ដ៏ អាក្រក់ បំផុត 
#ព្រះឣង្គ ទាំងអស់ ត្រូវ ធ្វើ ការបកស្រាយ និង ទំនុក បំរុង ដល់ ក្បត់ 

ព្រះឣង្គ ទាំងអស់ ត្រូវ ធ្វើ ការបកស្រាយ និង ទំនុក បំរុង ដល់ ក្បត់ 
  គឺ ក សាង ជិត ២៤ ថ្ងៃ ក្នុង ខែ គីឡូម៉ែត្រ ឆ្នាំ
ព្រះឣង្គ ប្រែ ចិត្ត ឲ្យ រួច ពី ការងារ អស់ ហើយ ហើយ ! 
  ពិតជា ល្អ បើ ប្រៀបធៀប ជាមួយ រឿង ដែល មិន មាន
ព្រះឣង្គ មាន ព្រះភាគ របស់ ក្រុម អ្នកជំនាញ ។ 
  ក្នុង ពេល ដែល មាន ក្រុម មនុស្ស ភាគ ច្រើន ពិគ្រោះ ជាមួយ ក្រុម ពាក់ព័ន្ធ នឹង
ព្រះឣង្គ ទ្រង់ ប្រទាន ព័ត៌មាន មិន ពិត មួយ ស្ថានភាព នៃ ព័ត៌មាន ក្លែងក្លាយ ផ្ទាល់ ព្រមទាំង ព័ត៌មាន ក្លែងក្លាយ និង រូបភាព ដ៏ អាក្រក់ បំផុត 
 
ព្រះឣង្គ ក្នុង ឱកាស រាជ វិវឌ្ឍ នៃ ប្រទេស ជប៉ុន ពេល ឡាយ បំពេញ បេសកកម្ម និង ចុះហត្ថលេខា លើ កិច្ចព្រមព្រៀង 
  សៀមរាប ៖ នៅ ថ្ងៃទី


In [0]:
TEXT = "លោក Trump"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#លោក Trump ប្រើ កងកម្លាំង យោធា មួយ ពាន់ 
#លោក Trump បង្កើត ក្រុម នៅ ប្រទេស ផ្សេង ៗ ដល់ អាមេរិក ដើម្បី ធ្វើ ប្រតិបត្តិការ ទៅ លើ ពិភពលោក 
#លោក Trump ត្រៀម ខ្លួន ដើម្បី ទទួល ជោគជ័យ 
#លោក Trump ឲ្យដឹង ថា ទីក្រុង វ៉ាស៊ីនតោន អាម៉េរិក ភាគ ច្រើន គឺ ស្ថិត នៅក្នុង តំបន់ អឺរ៉ុប ) ។ 

លោក Trump អះអាង ថា ប្រព័ន្ធ ការពារ សន្តិសុខ រដ្ឋ បាន ប្រើប្រាស់ ប្រព័ន្ធ ការពារ កញ្ចក់ Xr 
  ភ្នំពេញ ៖ នៅ ចុង ខែ នេះ
លោក Trump ចុះ ពិនិត្យ ស្ថានភាព សុខភាព របស់ ក្រុមហ៊ុន បរទេស នៅ កម្ពុជា 
  ( ភ្នំពេញ ) ៖ លោក Charles Peng ប្រធាន
លោក Trump ហេង ចំណេញ កំពុង រត់ ពី ហើយនិយាយ ទៅកាន់ សមាជិក ព្រឹទ្ធសភា 
  លោក Boris Johnson បាន ឱ្យដឹង
លោក Trump ឲ្យដឹង ថា ទីក្រុង វ៉ាស៊ីនតោន អាម៉េរិក ភាគ ច្រើន គឺ ស្ថិត នៅក្នុង តំបន់ អឺរ៉ុប ) ។ 
  ពួកគេ បាន ប្រកាស ថា នឹង
លោក Trump ទទួលមរណភាព នៅ ស្វាយរៀង និង ក្តៅ ក្នុង ពិភព ដង្ហើម រន្ទះ នៃ ចលនា អតីត យុទ្ធជន ខ្មែរ ក្រហម 
  ភ្នំពេញ ៖ លោក ប្រាយុទ្ធ


In [0]:
TEXT = "លោក ​ហ៊ុន សែន​"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#លោក ​ហ៊ុន សែន​ ឆ រដ្ឋមន្រ្ដី និង ជា ប្រធាន កិត្តិយស នៃ សាលា និង សាកលវិទ្យាល័យ ឯកជន ប្រចាំ តំបន់ រដ្ឋសភា ថា គម្រោង នោះ បាន ចុះ អន់
#លោក ​ហ៊ុន សែន​ ជា តារា សម្តែង ម្នាក់ ក្នុង កម្មវិធី Produce Strong Cup 
#លោក ​ហ៊ុន សែន​ ជា ប្រធាន មន្ទីរ បរិស្ថាន ខេត្ត ព្រះវិហារ នឹង ចូល ជា អនុ ប្រធាន មន្ទីរ បរិស្ថាន ស្រុក និង ពលករ ជនបទ 
#លោក ​ហ៊ុន សែន​ ប្តូរ លិខិត មក រដ្ឋមន្ត្រី ក្រសួង ការពារ ជាតិ 

លោក ​ហ៊ុន សែន​ ប្តូរ លិខិត មក រដ្ឋមន្ត្រី ក្រសួង ការពារ ជាតិ 
  ព័ត៌មាន លម្អិត ជុំវិញ ប្រជុំ សារព័ត៌មាន អាមេរិក The Wall Street
លោក ​ហ៊ុន សែន​ ៖ លោក ឲ្យ តាមដាន ឡើងវិញ ថា អ្វី ដែល កើតឡើង គឺ កើត ឡើង ដោយ ឥត គិត ថ្លៃ 
  ( ភ្នំពេញ )
លោក ​ហ៊ុន សែន​ ៖ " អ្នក ដែល មកពី ប្រទេស ស៊ុយអែត ត្រូវ ប្រជុំ សភា ៥ ជាន់ លើក ចុងក្រោយ នៅ កម្ពុជា ហើយ បាន ត្រឹមត្រូវ »
លោក ​ហ៊ុន សែន​ ៖ « យើង ឱ្យ ឧប្បត្តិហេតុ ហើយ មាន ប្រយោជន៍ ដល់ គ្រួសារ ក្រីក្រ » 
  ( ភ្នំពេញ ) ៖ លោក ទេសរដ្ឋមន្ត្រី ៖
លោក ​ហ៊ុន សែន​ បាន ជួប ជួបប្រទះ លោក រដ្ឋមន្ត្រី ការបរទេស ចិន លោកស្រី ស្វាយ យ៉ា ដែល កាន់ តំណែង ជា នាយក រដ្ឋមន្ត្រី កម្ពុជា នៅ ថ្ងៃទី ២៣


In [0]:
TEXT = "អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី ត្រូវបាន ចាប់ខ្លួន និង កាត់ទោស និង ឃុំខ្លួន នៅ ភ្នំពេញ 
#អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី និង បក្ខពួក ខ្លះ គាំទ្រ ពលរដ្ឋ ដែល កំពុង រង ការចោទប្រកាន់ ពី បទ ញុះញង់ របស់ អ្នក ខ្វះ ដីធ្លី 
#អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី សម រង្ស៊ី កំពុង បំពេញ ការងារ 


អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី ត្រូវបាន ចាប់ខ្លួន និង កាត់ទោស និង ឃុំខ្លួន នៅ ភ្នំពេញ 
  ( ភ្នំពេញ ) ៖ លោក ហ៊ុន សុខា ប្រធាន ការិយាល័យ ប្រឆាំង អំពើពុករលួយ
អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី កំពុង ច្រៀង និង មាន បទ « No Zone » ក្នុង រឿង « The Cambodian More
អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី បាន ចុះ ទៅ បំពេញ ទស្សនកិច្ច នៅ កម្ពុជា នៅ ទីក្រុង ភ្នំពេញ ដើម្បី ត្រៀម ខ្លួន ធ្វើជា សមាជិក ក្រុម ធាន អាន ។ 
 
អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី និង បក្ខពួក ខ្លះ គាំទ្រ ពលរដ្ឋ ដែល កំពុង រង ការចោទប្រកាន់ ពី បទ ញុះញង់ របស់ អ្នក ខ្វះ ដីធ្លី 
  ភ្នំពេញ ៖ លោក
អតីត ប្រធាន គណបក្ស ប្រឆាំង លោក សម រង្ស៊ី សម រង្ស៊ី កំពុង បំពេញ ការងារ 
  ភ្នំពេញ ៈ គណៈកម្មាធិការ ជាតិ រៀបចំ ការបោះឆ្នោត នៅ ថ្ងៃទី ២៥ ខែកក្កដា ឆ្នាំ ២០១៩ នេះ បាន


In [0]:
TEXT = "​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង ជើង ចាស់ ឆ្នើម របស់ កម្ពុជា ក្នុង បទ « One United » 
#​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង សម្ដែង នៅក្នុង ពិធី ខួប ៧ ឆ្នាំ បាន ចប់ ជាស្ថាពរ 
#​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង ច្រៀង ដោយ លោក ស៊ីន ស៊ី សាមុត ក ក្រើក 


​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង Serie Rey ចំនួន ៣០ លាន ឆ្នាំ មុន 
  ( ភ្នំពេញ ) ៖ តារា កិត្តិយស រូប នេះ បាន ធ្វើ
​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង ច្រៀង ដោយ លោក ស៊ីន ស៊ី សាមុត ក ក្រើក 
  ស៊ុន ចាន់ សាមុត ៖ ជា តារាចម្រៀង ច្រៀង ចម្រៀង 
  ដោយ ខាង
​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង និង ចម្រៀង ថ្មី នេះ ឃើញ ថា ជា លើក ទី ១ ក្នុង ប្រទេស កម្ពុជា 
  បរទេស ៖ ប្រិយមិត្ត ជាច្រើន បាន សម្តែង
​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង បទ មាន សម្លេង ពីរោះ ពីរោះ ពីរោះ ពីរោះ ច្រៀង ពីរោះ លើ លោក 
  ភ្នំពេញ ៖ និង ស្ងាត់ ៗ ពិតជា ពីរោះ តែម្តង
​អ្នក ​និពន្ធ​ បទ​ ចម្រៀង បទ « Time One » 
  ភ្នំពេញ ៖ Share នឹង ទទួលបាន ការគាំទ្រ ពី អ្នកគាំទ្រ ក្នុង តំបន់ ដែល


In [0]:
TEXT = "​លោក ស៊ីន ស៊ី សាមុត" #bad segmentation
N_WORDS = 15
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.95) for _ in range(N_SENTENCES)))
#​លោក ស៊ីន ស៊ី សាមុត មាន អ្នកគាំទ្រ ច្រើន ជាង Facebook Hazard ច្រៀង នៅ ខណៈ តូច ៗ ទាំង ពេល ថ្ងៃ 
#​លោក ស៊ីន ស៊ី សាមុត ព្យាយាម រក ប្រាក់ ព្យាបាល លើ មន្ទីរពេទ្យ 
#​លោក ស៊ីន ស៊ី សាមុត ទៅដល់ ខេត្ត ពោធិ៍សាត់ បញ្ជាក់ ពី កម្មវិធី រឿង ភ្ញាក់ផ្អើល ហើយ ! 
#​លោក ស៊ីន ស៊ី សាមុត ចំណាយ ពេល ត្រឹមតែ ១០០ នាទី ទៀត ទេ ? 


​លោក ស៊ីន ស៊ី សាមុត បាញ់ ផ្សែង ៣ គ្រាប់ នៅ ឋាន ព្រះច័ន្ទ .. 
  ភិក្ខុ នី ប៊ុន ធឿន របស់ សង្កាត់
​លោក ស៊ីន ស៊ី សាមុត កំពុង ដាក់ពាក្យបណ្តឹង ប្រកាស ឲ្យ ឃើញ ថា ៖ ផោន ជា ជរា តូច បំផុត រីឯ ម្សៅ
​លោក ស៊ីន ស៊ី សាមុត របស់ នាយ ទាហាន សកម្ម ភ័យខ្លាច ទម្ងន់ ទម្ងន់ 110 គីឡូក្រាម 
  ព្រះវិហារ ៖ នាយ ឧត្តមសេនីយ៍ ឈុំ
​លោក ស៊ីន ស៊ី សាមុត ព្រម ទទួល ការលំបាក ជាខ្លាំង ក្នុង កម្មវិធី ជ្រើសរើស សម្ព័ន្ធ ដណ្ដើម ឯករាជ្យ ជាក់ស្តែង ក្នុង ការជ្រើសរើស អចិន្រ្តៃយ៍ 
 
​លោក ស៊ីន ស៊ី សាមុត ឃើញ ថា បែប នេះ ពិសេស ! 
  តារា សម្ដែង ជួរ មុខ ថៃ បច្ចុប្បន្ន កញ្ញា សាន


In [0]:
TEXT = "និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន ១៣ រូប
#និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន ៤ រូប ក្នុង ចំណោម ចំនួន ២៥ នាក់
#និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន ៨០ លាន នាក់ ជា ក្រុម អ្នក រួម ជាតិ កម្ពុជា 

និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន វិល មក ចូលរួម ការសិក្សា នៅ សកលវិទ្យាល័យ អាស៊ី អាគ្នេយ៍ 
  ( ភ្នំពេញ ) ៖ លោក ឧត្តមសេនីយ៍ ឯក ហ៊ុន ម៉ាណែត អគ្គ ស្នងការ
និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន ៦ ដែល កំពុង រស់ នៅ សម័យ សង្គ្រាម ត្រជាក់ ចាប់ផ្ដើម ។ 
  នៅ ថ្ងៃ នេះ លោក បណ្ឌិត អ៊ិត សំហេង បាន ថ្លែង
និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន ៤ រូប ក្នុង ចំណោម ចំនួន ២៥ នាក់ 
  ភ្នំពេញ ៖ នា ព្រឹក ថ្ងៃទី ២៥ ខែ កក្កដា ឆ្នាំ ២០១៩ នេះ ក្រសួង
និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន ៨០ លាន នាក់ ជា ក្រុម អ្នក រួម ជាតិ កម្ពុជា 
  ( ភ្នំពេញ ) ៖ បន្ទាប់ពី ក្រុម ស្ថាបនិក គណៈកម្មាធិការ ចិន នៅ
និសិ្សត បញ្ញាវ័ន្ត ខ្មែរ ចំនួន ៦៨ នាក់ ស្មើនឹង ជាង ១០០ នាក់ 
  ភ្នំពេញ ៖ បន្ទាប់ពី មាន ការរិះគន់ ពី លោក នាយក រដ្ឋមន្រ្តី ហ៊ុន សែន បាន បន្ដ


## Categorization

### Copy accident data file from github

In [0]:
filename = 'accident_docs.pkl';

In [15]:
import urllib.request
url = 'https://github.com/phylypo/khmer-language-model-ulmfit/blob/master/data/accident_docs.pkl?raw=true'
print("url", url)
urllib.request.urlretrieve(url, filename)

url https://github.com/phylypo/khmer-language-model-ulmfit/blob/master/data/accident_docs.pkl?raw=true


('accident_docs.pkl', <http.client.HTTPMessage at 0x7f1d78da6eb8>)

In [17]:
df2 = pd.read_pickle(filename);
df2

,<built-in function id>,text,cat
0,5279,ចាប់ពី ថ្ងៃ នេះ តទៅ YouTube លែង ឲ្យ បង្ហោះ វ...,no_accident
1,5831,មក ស្គាល់ រុក្ខជាតិ ម្យ៉ាង នៅក្នុង សមុទ្រ កម្...,no_accident
2,6385,មក មើល កម្មករ ខ្សាច់ សាច់ដុំ ធំ ៗ នៅ កាមេរ៉ូន...,no_accident
3,7452,ដោយសារតែ បើកបរ លឿន ជិត ៗ គ្នា ពេក រថយន្ត ៦ គ្...,accident
4,7474,គ្មាន ទៀត ទេ រថយន្តក្រុង ប្រើ សាំង ! សៀង ហៃ ក...,no_accident
...,...,...,...
495,22646,ទម្រង់ ចង្កា អាច បង្ហាញ ពី អត្តចរិត និង ជោគវាស...,no_accident
496,22647,ជា នារី ដែល គួរឲ្យ ស្រឡាញ់ និង មើលទៅ មាន ទំនុ...,no_accident
497,22688,Pub Street នឹង ប្រែ មុខមាត់ ថ្មី ជាមួយនឹង ក...,no_accident
498,22776,អង្គការ អូតង់ រៀបចំ ចំណាត់ការ ទប់ទល់ នឹង ការប...,no_accident


In [0]:
df2['label'] = np.where(df2['cat'] == 'accident', 1, 0)
df = df2
df

,<built-in function id>,text,cat,label
0,5279,ចាប់ពី ថ្ងៃ នេះ តទៅ YouTube លែង ឲ្យ បង្ហោះ វ...,no_accident,0
1,5831,មក ស្គាល់ រុក្ខជាតិ ម្យ៉ាង នៅក្នុង សមុទ្រ កម្...,no_accident,0
2,6385,មក មើល កម្មករ ខ្សាច់ សាច់ដុំ ធំ ៗ នៅ កាមេរ៉ូន...,no_accident,0
3,7452,ដោយសារតែ បើកបរ លឿន ជិត ៗ គ្នា ពេក រថយន្ត ៦ គ្...,accident,1
4,7474,គ្មាន ទៀត ទេ រថយន្តក្រុង ប្រើ សាំង ! សៀង ហៃ ក...,no_accident,0
...,...,...,...,...
495,22646,ទម្រង់ ចង្កា អាច បង្ហាញ ពី អត្តចរិត និង ជោគវាស...,no_accident,0
496,22647,ជា នារី ដែល គួរឲ្យ ស្រឡាញ់ និង មើលទៅ មាន ទំនុ...,no_accident,0
497,22688,Pub Street នឹង ប្រែ មុខមាត់ ថ្មី ជាមួយនឹង ក...,no_accident,0
498,22776,អង្គការ អូតង់ រៀបចំ ចំណាត់ការ ទប់ទល់ នឹង ការប...,no_accident,0


In [0]:
train_df = df[0:400]
test_df = df[400:]

In [0]:
# correct mislabels -- see these info in interpretation section
df.at[76, 'label'] = 0
df.at[246, 'label'] = 0
df.at[140, 'label'] = 1
train_df.at[76, 'label'] = 0
train_df.at[246, 'label'] = 0
train_df.at[140, 'label'] = 1
train_df.at[22, 'label'] = 1
train_df.at[29, 'label'] = 1

In [0]:
print(train_df['text'][395])
print(train_df['label'][395])

អ្នក ជិះ បញ្ច្រាស រង្វង់ មូល វិមាន ឯករាជ្យ នៅតែ បន្ត មាន ,  ក្នុង យប់ មិញ  សមត្ថកិច្ច ឃាត់ បាន យានយន្ត ចំនួន ៩ គ្រឿង ថែម ទៀត  ( ភ្នំពេញ ) ៖  អ្នក ជិះ បញ្ច្រាស នៅ រង្វង់ មូល វិមាន ឯករាជ្យ  នៅតែ បន្ត កើតមាន ស្ទើរ ជា រៀងរាល់ ថ្ងៃ  បើ ទោះបីជា សមត្ថកិច្ច  និង អាជ្ញាធរ ដាក់ កម្លាំង ប្រចាំ ការស្ទាក់ ចាប់  ទាំង យប់ ទាំង ថ្ងៃ ក្តី  ជាក់ស្តែង នៅ យប់ មិញ ថ្ងៃទី ១០  ខែកុម្ភៈ  ឆ្នាំ ២០១៩  សមត្ថកិច្ច អនុវត្តន៍ ច្បាប់  និង អាជ្ញាធរ បាន ឃាត់ បាន  យានយន្ត ចំនួន ០៩ គ្រឿង  បញ្ជូន ទៅ ស្នងការដ្ឋាន នគរបាល រាជធានី ភ្នំពេញ ។ របាយការណ៍ របស់ អាជ្ញាធរ ខណ្ឌ ដូនពេញ  បាន ឲ្យដឹង ថា  នៅ ថ្ងៃទី ១០  ខែកុម្ភៈ  ឆ្នាំ ២០១៩  ចាប់ពី ម៉ោង ០៧៖០០ នាទី ព្រឹក  ដល់ ម៉ោង ១០៖០០ នាទី យប់  កម្លាំង គណៈ បញ្ជាការ ឯកភាព ខណ្ឌ ដូនពេញ  សហការ ជាមួយ ការិយាល័យ ចរាចរណ៍ ជើង គោក  នៃ ស្នងការដ្ឋាន នគរបាល រាជធានី ភ្នំពេញ  អនុវត្ត យុទ្ធនាការ ឃាត់ ម៉ូតូ  និង រថយន្ត  បើក បញ្ច្រាស ទិសដៅ ចរាចរណ៍  នៅ រង្វង់ មូល ជុំវិញ វិមាន ឯករាជ្យ  និង ផ្លូវ ព្រះសុរាម្រឹត  ស្ថិត ក្នុង សង្កាត់ ចតុមុខ  ខណ្ឌ ដូនពេញ  ឃាត់ យានយន្ត សរុប ចំនួន ០៩ គ្រឿង  ក្នុង នោះ រួមមាន៖  ម៉ូតូ

In [0]:
#train_df
train_df.loc[train_df['label'] == 1] #130/400

,<built-in function id>,text,cat,label
3,7452,ដោយសារតែ បើកបរ លឿន ជិត ៗ គ្នា ពេក រថយន្ត ៦ គ្...,accident,1
5,7513,រថយន្ត តារា កំប្លែង ជើង ចាស់ នាយ កុយ បែក កង់...,accident,1
6,7516,ជនជាតិ យូក្រែន ម្នាក់ ក្រឡាប់ រថយន្ត នៅ ស្រុក...,accident,1
9,7754,បុរស ម្នាក់ បាន ស្លាប់ បាត់បង់ជីវិត ភ្លាម ៗ ...,accident,1
10,7765,ឲ្យ ក្មេង មិន ទាន់ គ្រប់ អាយុ បើក ឡាន បុក គេ ...,accident,1
...,...,...,...,...
392,19990,សូម កុំ បើកបរ ហួស ល្បឿន កំណត់ ! ស្លាប់ ១០ នាក...,accident,1
393,20002,យុទ្ធនាការ រឹតបន្ដឹង ច្បាប់ ចរាចរណ៍ ទូទាំង ប្រ...,accident,1
394,20004,ស្លាប់ ១០ នាក់ និង របួស ២១ នាក់ ក្នុង គ្រោះថ...,accident,1
396,20052,ម៉េច ចឹង ! គូស្នេហ៍ មួយ គូ នេះ ឡើង ផ្អើល គេ ឯ...,accident,1


In [0]:
test_df.loc[test_df['label']==1] # 41/100

,<built-in function id>,text,cat,label
403,20150,ជិះ ម៉ូតូ ឆ្លងកាត់ ផ្លូវ ខ្វះ ការប្រុងប្រយ័ត្ន...,accident,1
407,20230,ពន្លះ ឡើង ខ្ទេច ហើយ រត់ មាន អី បងប្អូន ប្រជាព...,accident,1
409,20301,រថយន្ត បែន ដឹក ដី បើក បុក រថភ្លើង ធ្លាក់ ក្ប...,accident,1
412,20332,អ្នកបើក ឡាន ធំ ត្រូវតែ ដឹង ... ចៀស កើត ហេតុការ...,accident,1
413,20334,រន្ធត់ ! ជនជាតិ កូរ៉េ បើក រថយន្ត ឡិចស៊ីស RX3...,accident,1
414,20376,រថភ្លើង និង រថយន្ដ បុក គ្នា នៅ ស្រុក ព្រៃ នប...,accident,1
415,20386,ទៀត ... រថភ្លើង និង ឡាន ជល់ គ្នា យ៉ាង ញាក់ សាច...,accident,1
416,20449,រថយន្ត បែន កង់ ១០ មួយ គ្រឿង បើក ក្នុង ល្បឿន ល...,accident,1
417,20451,រថយន្ដ បែន ដឹក ដី មួយ គ្រឿង បុក គ្នា ជាមួយ រថ...,accident,1
418,20455,រថភ្លើង និង រថយន្ដ បុក គ្នា នៅ ស្រុក ព្រៃ នប...,accident,1


## Sentiment analysis

### Language model

In [0]:
df = pd.concat([train_df,test_df], sort=False)

In [0]:
path = Path('kmwiki_data')
#path = Path('drive/My Drive/khmer_ulmfit')

#!ls "drive/My Drive/khmer_ulmfit/models/km_wt_vocab.pkl"
#data_lm
#path.ls()

In [0]:
data_lm = (TextList.from_df(df, path, cols='text')
    .split_by_rand_pct(0.1, seed=42) #0.1 to 0.2
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [0]:
lr = 1e-3
lr *= bs/48

In [0]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))
#0	5.595953	4.762003	0.195926	00:05
#1	5.117899	4.595739	0.206473	00:05
#5k crf
#0	4.323324	3.677873	0.313185	00:03
#1	4.132768	3.627848	0.318512	00:03
#1k crf 20% test
#0	4.839848	4.089501	0.268036	00:06
#1	4.562537	4.006660	0.276548	00:06

epoch,train_loss,valid_loss,accuracy,time
0,4.835725,4.126636,0.270893,00:06
1,4.544458,4.041156,0.276548,00:07


In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, lr, moms=(0.8,0.7)) #18 total
#0	4.542891	4.241218	0.245313	00:06
#7	3.951082	3.815118	0.298493	00:07
#18	3.416171	3.504143	0.335156	00:06
#5k crf
#0	3.974394	3.611540	0.322202	00:04
#9	3.568427	3.423896	0.340119	00:04
#1k crf with 20% test
#0	4.371777	3.981277	0.277917	00:08
#9	3.863489	3.708247	0.308304	00:08

epoch,train_loss,valid_loss,accuracy,time
0,4.370144,4.009106,0.279286,00:09
1,4.326116,3.960132,0.282560,00:09
2,4.252964,3.903078,0.290536,00:09
3,4.159999,3.854772,0.297202,00:09
4,4.070383,3.796848,0.301786,00:08
5,3.986990,3.763078,0.306845,00:08
6,3.924317,3.728368,0.308690,00:09
7,3.880824,3.720660,0.310417,00:09
8,3.847347,3.724601,0.311548,00:09
9,3.846570,3.717937,0.311369,00:09


In [0]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Test model

In [0]:
TEXT = "អ្នក ជិះ"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#អ្នក ជិះ ម៉ូតូ បុក រះ កង់ នោះ ហើយ បង្កឲ្យ មាន គ្រោះថ្នាក់ ចរាចរណ៍ រវាង រថយន្ត បុក គ្នា និង ស្លុតចិត្ត រង ការខូចខាត ។ 

អ្នក ជិះ ពីក្រោយ បើក ទ្វារ ផ្លូវ ជាតិ លេខ ៤ និង ផ្លូវ ជាតិ លេខ ៣ លេខ ២ តាម បណ្តោយ ផ្លូវ ជាតិ លេខ ៧
អ្នក ជិះ ម៉ូតូ រត់ តាម ផ្លូវ ជាតិ លេខ ៣ ត្រង់ ចំណុច ស្ពាន ឈើ ៣ ខ្នង បណ្តាលឱ្យ ចាប់ផ្តើម ជិត រកឃើញ បែប នេះ !
អ្នក ជិះ ម៉ូតូ មិន តិច ជាង ២ គ.ម និង រថយន្ត ភ្នែក ចាប់ លេខ ២៥ និង ទាញ ត ទៅ តាម ផ្លូវ ច្បាប់ នៃ
អ្នក ជិះ ទោ ដែល មាន ទិសដៅ ក្នុង ចំណោម ជន បង្ក ជា អ្នក មាន លទ្ធភាព តិចតួច ក្នុង ការធ្វើ អត្តឃាត ជំនាញ : របាយការណ៍ របស់
អ្នក ជិះ ម៉ូតូ ជាន់ ខ្ពស់ ៖ បុរស ម្នាក់ បើក រថយន្ដ ធ្វើអោយ អ្នកបើកបរ ម៉ូតូ ២ គ្រឿង បុក ចូល ផ្លូវ តែ ម្តង ៖ រថយន្ត


In [0]:
TEXT = "ស្រែក យំ"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#ស្រែក យំ រៀបរាប់ ថា អោយ ដី នៅ ឆ្ងាយ អាច ដោះ ចេញ ពី ដី ខ្សាច់ បាន ។

ស្រែក យំ ថា យក កាំភ្លើង ។ ឃើញ ថា កាំភ្លើង បាញ់ គ្រាប់ កាំភ្លើង តូច មួយ គ្រាប់ តែ ឈានដល់ បាក់ ស្រែ ។ ដោយ មូលហេតុ
ស្រែក យំ ខ្លាំង ម្ល៉េះ ! ប្អូន ប្រុស ម្នាក់ ស្លាប់ ទៅ ហើយ ធម្មតា បាន ដួល សន្លប់ ក្នុង ថ្លុក ឈាម ហើយ បណ្តាលឲ្យ ស្លាប់ ភ្លាម
ស្រែក យំ ថា « ខ្ញុំ លេង » ក៏ មាន ឈ្មោះ ថា « កម្ពុជា » បាន នាំគ្នា វាយ ខ្នោះ ផ្ទះ ម្នាក់ នៅ ខេត្ត
ស្រែក យំ ប៉ូលិស ដេញ តាម មក ពីក្រោយ ក៏ រត់ មិន រួច ស្រាប់តែ មាន ម៉ូតូ ខ្មែរ ដក ទឹក ( លាង របួស ) ហើយ
ស្រែក យំ … ស្នាម ជើង សៀមរាប ៖ រង ការខូចខាត យ៉ាង ដំណំ , មិន នឹកស្មាន ថា ករណី ឃាតកម្ម នោះ ត្រូវបាន គេ សម្លាប់ និង


In [0]:
TEXT = "ក្រុម ចោរ"
N_WORDS = 20
N_SENTENCES = 5
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
#ក្រុម ចោរ លួច យក ប្រាក់ ចំណេញ បាន យក ទៅ លក់ នៅ ផ្ទះ ជ្រៃ ឃុំ សង្កែ ស្រុក អង្គរ បូរី ខេត្ត កណ្តាល ។

ក្រុម ចោរ យក ជញ្ជីង ថ្លឹង លុយ មក លក់ លក់ នៅ កម្ពុជា ទិញ ដី ហើយ ចូល ផ្ទះ ប្រជាពលរដ្ឋ នៅ ក្រុង តាខ្មៅ ខេត្ត កណ្ដាល
ក្រុម ចោរ លួច ទ្រព្យសម្បត្តិ យ៉ាង ច្រើន តែ មក ធ្វើ " លួច ប្លន់ » នៅ ស្រុក សេសាន ខេត្ត ស្វាយរៀង ៖ មន្ត្រី យោធា ម្នាក់
ក្រុម ចោរ លួច ១០០ នាក់ រត់ ទៅ វិញ បត់ គឺ ជន គល់ ផ្លូវ ស្រុក ចំនួន ៣ នាក់ បាន ស្លាប់ ដោយសារ អំពើ
ក្រុម ចោរ លួច ឡាន នៅក្នុង បន្ទប់ ចង ក ខេត្ត ក្រចេះ ៖ បុរស ម្នាក់ ត្រូវបាន សមត្ថកិច្ច ឃាត់ខ្លួន នៅ ក្លិប កម្សាន្ត « មុខ
ក្រុម ចោរ លួច ឃើញ ស្ត្រី ជា ប្ដី ម្នាក់ បាន រត់ ទៅ កាប់ ជាមួយ ស្ត្រី ម្នាក់ បាន ស្លាប់ បង្ក ជា ការភ្ញាក់ផ្អើល នៅ ប៉ុស្តិ៍


In [0]:
TEXT = "អត្តសញ្ញាណ"
N_WORDS = 50
N_SENTENCES = 5
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.2) for _ in range(N_SENTENCES)))

អត្តសញ្ញាណ របស់ អ្នក មាន ដំណើរ ក្នុង រឿង នេះ ។ បើ អ្នក ចង់ ដឹង ថា អ្នក មាន ចិត្ត ល្អ មែន ទេ អ្នក នឹង អាច ធ្វើ បាន ។ បើ អ្នក ចង់ ដឹង ថា អ្នក ពិតជា មាន ចិត្ត ល្អ មែន ទេ អ្នក នឹង អាច ធ្វើ អ្វី បាន ។ បើ អ្នក ចង់ ធ្វើ អ្វី ៗ
អត្តសញ្ញាណ របស់ គាត់ មាន ឈ្មោះ ថា " ម " ( ជា ភាសា អង់គ្លេស ) ។ លោក ទោ Sports Cars បាន បញ្ជាក់ ថា នៅ មុនពេល កើត ហេតុ គេ ឃើញ មាន ករណី គ្រោះថ្នាក់ ចរាចរណ៍ មួយ ដែល មាន ឈ្មោះ ថា Beat Cars Team Team Cars
អត្តសញ្ញាណ របស់ អ្នក មាន ជំងឺ នេះ មាន ឈ្មោះ ថា ស្រី ស្រី ។ ដោយឡែក អ្នក ទាំង ពីរ មាន ឈ្មោះ ថា ស្រី ស្រី ។ ស្ត្រី ជា ប្តី អាយុ ២០ ឆ្នាំ មាន ផ្ទៃពោះ នៅ ភូមិ តា ធំ ឃុំ ត្រពាំង ភ្លាំង ស្រុក ព្រៃ នប់ ខេត្ត ក្រចេះ ។ ជនរងគ្រោះ ជា ប្តី អាយុ ១៩ ឆ្នាំ មុខរបរ កសិករ
អត្តសញ្ញាណ របស់ អ្នក មាន អាយុ ប្រហែល ជាង ១០០ ឆ្នាំ មាន ទីលំនៅ ភូមិ តា ធំ ឃុំ ត្រពាំង ធំ ស្រុក សំបូរ ខេត្ត ក្រចេះ ។ ជនរងគ្រោះ ឈ្មោះ ស៊ន សុខ ភេទ ប្រុស អាយុ ៣៥ ឆ្នាំ មុខរបរ កម្មករ សំណង់ ផ្ទះ ស្នាក់ នៅ ភូមិ ព្រែក ព្នៅ ឃុំ ត្រពាំង វែង ស្រុក ស្នួល ខេត្ត ក្រចេះ ។ ជនរងគ្រោះ ឈ្មោះ សាន
អត្តសញ្ញាណ របស់ អ្នក មាន ច្រើន ។ អ្នក ដែល មាន ឈ្មោះ

### Classifier

In [0]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(0.1, seed=42) #0.1 to 0.2
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [0]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [0]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [0]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [0]:
lr=2e-2
lr *= bs/48

In [0]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))
#0	0.602139	0.537407	0.675000	0.315789	00:01
#1	0.448829	0.470891	0.750000	0.545455	00:01
#5k crf
#0	0.391408	0.377960	0.937500	0.921212	00:02
#1	0.279988	0.414496	0.825000	0.730769	00:02
#1k crf
#0	0.484861	0.479978	0.850000	0.824242	00:01
#1	0.378260	0.499441	0.750000	0.573427	00:01

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.456686,0.351467,0.950000,0.933333,00:01
1,0.332043,0.347622,0.875000,0.814815,00:01


In [0]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))
#0	0.211190	0.208395	0.975000	0.969697	00:01
#1	0.215615	0.378002	0.750000	0.545455	00:01
#5k crf
#0	0.168557	0.346563	0.887500	0.848276	00:02
#1	0.159936	0.488095	0.825000	0.740741	00:02
#1k crf
#0	0.292124	0.507463	0.762500	0.596014	00:01
#1	0.263080	0.487999	0.787500	0.619048	00:01

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.200325,0.219820,0.925000,0.903226,00:01
1,0.195049,0.158604,0.950000,0.933333,00:01


In [0]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))
#0	0.229428	0.430638	0.750000	0.545455	00:01
#1	0.268126	0.531731	0.675000	0.315789	00:01
#5k crf
#0	0.115307	0.295327	0.900000	0.866071	00:02
#1	0.135331	0.383951	0.887500	0.848276	00:02
#1k crf datafix
#0	0.201699	0.054135	1.000000	1.000000	00:01

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.201699,0.054135,1.000000,1.000000,00:01
1,0.160199,0.039080,1.000000,1.000000,00:01


In [0]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))
#0	0.148446	0.170359	0.925000	0.896552	00:01
#1	0.142438	0.132280	0.975000	0.967742	00:01
#5k crf
#0	0.135274	0.300655	0.925000	0.902821	00:03
#1	0.089168	0.309268	0.925000	0.902821	00:03

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.130061,0.033847,1.000000,1.000000,00:02
1,0.090500,0.044649,0.975000,0.967742,00:02


In [0]:
learn_c.unfreeze()
learn_c.fit_one_cycle(10, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))
#0	0.063082	0.238846	0.920000	0.894737	00:02
#9	0.034720	0.356180	0.900000	0.871795	00:02

#0	0.083462	0.109984	0.975000	0.967742	00:02
#9	0.053348	0.073645	0.975000	0.967742	00:02
#learn_c.fit_one_cycle(10, slice(lr/(2.6**4),lr), moms=(0.7,0.5))
# with fix dataset we can get 100%
#0	0.050068	0.038671	1.000000	1.000000	00:02
#9	0.033688	0.017339	1.000000	1.000000	00:02

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.055295,0.039217,0.975000,0.967742,00:02
1,0.056032,0.038041,0.975000,0.967742,00:02
2,0.049582,0.042739,0.975000,0.967742,00:02
3,0.047049,0.021697,1.000000,1.000000,00:02
4,0.044079,0.018514,1.000000,1.000000,00:02
5,0.040420,0.013629,1.000000,1.000000,00:02
6,0.039288,0.023180,1.000000,1.000000,00:02
7,0.036255,0.030007,0.975000,0.967742,00:02
8,0.035607,0.024957,0.975000,0.967742,00:02
9,0.034302,0.026983,0.975000,0.967742,00:02


In [0]:
learn_c.unfreeze()
learn_c.fit_one_cycle(5, slice(lr/10/(2.6**4),lr/10), moms=(0.9,0.8))
#0	0.030002	0.314712	0.937500	0.918864	00:02
#4	0.031145	0.314854	0.937500	0.918864	00:02
#5k crf
#0	0.023964	0.369878	0.937500	0.918864	00:04
#4	0.022802	0.384393	0.937500	0.918864	00:04
#1k crf (Same perf a 5k) - repeat run still same perf (diff moms no effect)
#0	0.030585	0.357210	0.937500	0.918864	00:02
#4	0.024678	0.375762	0.937500	0.918864	00:02

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.017855,0.020872,0.975000,0.967742,00:02
1,0.019455,0.038736,0.975000,0.967742,00:02
2,0.023084,0.053620,0.975000,0.967742,00:02
3,0.024937,0.050382,0.975000,0.967742,00:02
4,0.022781,0.051458,0.975000,0.967742,00:02


In [0]:
learn_c.save(f'{lang}clas')
#learn_c
#Path: drive/My Drive/khmer_ulmfit; "km_textlist_class"

In [0]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)
# best: (tensor(0.9750), tensor(0.9677)) -- first time on 10% test set
#without crf data
#(tensor(0.9375), tensor(0.9206)) -- with 5K crf data
#(tensor(0.9750), tensor(0.9677))

(tensor(1.), tensor(1.))

### Interpretation

In [0]:
import matplotlib.cm as cm

txt_ci = TextClassificationInterpretation.from_learner(learn_c)
test_text = "គ្រោះថ្នាក់ ចរាចរ រវាង រថយន្ត ធុន ធំ និង ម៉ូតូ បណ្តាលឲ្យ បុរស ម្នាក់ ស្លាប់ " # "បុរស ម្នាក់ បាន ស្លាប់ បើក បុក រះ រថយន្ត និង ម៉ូតូ ជាច្រើន គ្រឿង" #"កម្មករ បើក រថយន្ត ចេញ ពី លាង បុក រុញ គូទ"
txt_ci.show_intrinsic_attention(test_text,cmap=cm.Purples)

/pytorch/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [0]:
txt_ci.show_top_losses(15)

Text,Prediction,Actual,Loss,Probability
xxbos យុវជន ពីរ នាក់ របួស ធ្ងន់ ស្រាល ដោយ ជិះ ម៉ូតូ មិន បាន ប្រុងប្រយ័ត្ន ខេត្ត រតនគិរី ៖ យុវជន ២ នាក់ បាន រងរបួស ធ្ងន់ ស្រាល បន្ទាប់ពី ជិះ ម៉ូតូ បុក គ្នា យ៉ាង ពេញ ទំហឹង ក្នុង ទិសដៅ បញ្ច្រាស គ្នា ។ ករណី គ្រោះថ្នាក់ ចរាចរណ៍ បាន កើតឡើង នៅ វេលា ម៉ោង ១១ និង ២៥ នាទី ព្រឹក ថ្ងៃព្រហស្បតិ៍ ៣ កើត ខែ មាឃ ឆ្នាំ ច សំរឹទ្ធិ ស័ក ព.ស ២៥៦២ ត្រូវ និង ថ្ងៃទី ៧ ខែកុម្ភៈ ឆ្នាំ ២០១៩ នៅ ចំណុច កែង,1,1,0.27,0.77
xxbos រថយន្ត បើក បុក រ៉ឺម៉ក ម៉ូតូ អេត ចាយ ១ គ្រឿង ពីក្រោយ ក្រឡាប់ កិន ពីលើ ស្លាប់ ម្នាក់ កំពង់ឆ្នាំង ៖ បុរស វ័យ ចំណាស់ ម្នាក់ បាន ស្លាប់ ភ្លាម ៗ ជាប់ ក្រោម រថយន្ត ជាមួយ រ៉ឺម៉ក ម៉ូតូ របស់ គាត់ ខណៈ ដែល មាន រថយន្ត ដឹក កុងតឺន័រ មួយ គ្រឿង បាន បើកបរ ក្នុង xxunk xxunk បុក រ៉ឺម៉ក ម៉ូតូ ពីក្រោយ ក្នុង ទិស ស្រប ទិស គ្នា ហើយ រថយន្ត បាន ក្រឡាប់ កិន ពីលើ រ៉ឺម៉ក ម៉ូតូ ។ ហេតុការណ៍ គ្រោះថ្នាក់ ចរាចរណ៍ ខាង លើ,1,1,0.17,0.85
xxbos លទ្ធផល ត្រួតពិនិត្យ ការអនុវត្ត ច្បាប់ ចរាចរណ៍ ថ្ងៃទី ៨ កុម្ភៈ យានយន្ត ល្មើស ៨១៧ គ្រឿង ភ្នំពេញ ៖ យោង តាម របាយការណ៍ ពី នាយកដ្ឋាន ចរាចរណ៍ និង សណ្ដាប់ធ្នាប់ សាធារណៈ នៃ អគ្គ ស្នងការដ្ឋាន នគរបាល ជាតិ ប្រាប់ ឲ្យដឹង ថា លទ្ធផល ត្រួតពិនិត្យ ការអនុវត្ត ច្បាប់ ចរាចរណ៍ ផ្លូវគោក គិត ចាប់ វេលា ម៉ោង ១៤ និង ០០ នាទី ថ្ងៃទី ៧ ខែកុម្ភៈ ឆ្នាំ ២០១៩ ដល់ វេលា ម៉ោង ១៤ និង ០០ នាទី ថ្ងៃទី ៨ ខែកុម្ភៈ ឆ្នាំ ២០១៩ xxunk បាយ យានយន្ត ល្មើស សរុប ចំនួន ៨១៧,1,1,0.12,0.89
xxbos xxmaj xxunk ៖ គ្រោះថ្នាក់ ចរាចរ រថយន្ត បែន ដឹក គ្រួស បើក បុក រះ រថយន្ត និង ម៉ូតូ ជាច្រើន គ្រឿង រួច រ៉េ បុក ចូល ផ្ទះ លក់ ចាប់ ហ៊ួយ បណ្តាលឲ្យ ស្លាប់ មនុស្ស ស្រី ម្នាក់ និង មនុស្ស ប្រុស ស្រី ៤ នាក់ របួស ធ្ងន់ - ព័ត៌មាន - xxup tvfb ក្រុង ព្រះសីហនុ៖ ( ដំណឹង បន្ថែម ហេតុការណ៍ គ្រោះថ្នាក់ ចរាចរណ៍ រថយន្ត ធំ បុក រះ នៅ ក្រុង ព្រះសីហនុ ) លោក ឧត្តមសេនីយ៍ ត្រី អ៊ី សុខា ស្នងការ រង ទទួល ផែន សណ្តាប់ធ្នាប់,1,1,0.11,0.89
xxbos មើល ម៉ូតូ ចោរ ដ៏ ថ្មី សន្លាង ... ឆក់ xxunk នាំគ្នា រត់ xxunk អាយុ បាត់ រាជធានី ភ្នំពេញ ៖ ម៉ូតូ ហុងដា សេ ១២៥ មួយ គ្រឿង ពណ៌ ខ្មៅ ស៊េរី ឆ្នាំ ២០១៩ មិន ទាន់ មាន ផ្លាកលេខ នៅ ឡើយ គឺ នៅ ថ្មី សន្លាង ដែល ទំនងជា ទើបនឹង ទិញ បាន ត្រូវ សមត្ថកិច្ច ដកហូត យក ទៅ រក្សាទុក ក្រោយពី ម្ចាស់ វា ២ នាក់ ធ្វើ សកម្មភាព ឆក់ កាបូប xxunk ជន បរទេស ម្នាក់ នៅ ម្តុំ xxunk ទួល xxunk តែ មិន,0,0,0.10,0.90
xxbos គិត អ្វី លែង ចេញ ! អស់ ២០ ម៉ឺន ដុល្លារ ព្រោះ ប្រុស មាន ល្បិច ផ្អើល លើ ស្ពាន ជ្រោយ ចង្វារ រាជធានី ភ្នំពេញ ៖ ស្ត្រី ម្នាក់ ជា អ្នកចង ការលុយ នៅ ចោម ចៅ ២ មាន វិបត្តិ ចាញ់បោក បុរស ម្នាក់ នៅ ក្រៅ ប្រទេស អស់ លុយ ២០ ម៉ឺន ដុល្លារ ក៏ សម្រេចចិត្ត ទៅ លោត ទឹក សម្លាប់ ខ្លួន នៅលើ ស្ពាន ជ្រោយ ចង្វារ ត្រូវ សមត្ថកិច្ច ជួយសង្គ្រោះ ទាន់ ពេលវេលា បង្កឱ្យ មាន ការភ្ញាក់ផ្អើល កាលពី ម៉ោង xxunk និង xxunk នាទី រំលង,0,0,0.07,0.93
xxbos ជិះ ម៉ូតូ ជិត ២០ គ្រឿង ប្រដាប់ ដោយ កាំបិត និង ដាវ សាម៉ូរ៉ៃ ដេញ កាប់ យុវជន ម្នាក់ រងរបួស ធ្ងន់ ដេក ក្នុង ថ្លុក ឈាម ភ្នំពេញ ៖ យុវជន ម្នាក់ ក្នុង ចំណោម xxunk ចំនួន ៣ នាក់ ត្រូវបាន ស្ទាវ មួយ ក្រុម មាន គ្នា រាប់ សិប នាក់ ជិះ ម៉ូតូ ជិត ២០ គ្រឿង ប្រដាប់ ដោយ កាំបិត និង ដាវ សាម៉ូរ៉ៃ ដេញ កាប់ បណ្តាលឲ្យ រងរបួស ធ្ងន់ ដេក ដួល ក្នុង ថ្លុក ឈាម ហើយ ត្រូវបាន រថយន្ត សាមុយ ដឹក បញ្ជូន ទៅកាន់ មន្ទីរពេទ្យ,0,0,0.07,0.93
xxbos xxunk មិន ដែល ភ្លេច កុក ! ចូល លួច ទូរស័ព្ទ ដៃ ក្នុង ហាង គេចខ្លួន បាន ៤ ថ្ងៃ នឹកស្មាន គេ មិន ដឹង ចុងក្រោយ ជាប់ ខ្នោះ សមត្ថកិច្ច ! - ព័ត៌មាន - xxup tvfb ភ្នំពេញ៖ ប៉ុស្តិ៍ រដ្ឋបាល ឃ្មួញ បាន ធ្វើ ការឃាត់ខ្លួន ជនសង្ស័យ ចំនួន ០២ នាក់ ពាក់ព័ន្ធ នឹង ករណី អំពើ លួច ( ឆ្កឹះ សោ ចូល លួច យក ទូរស័ព្ទដៃ ) កាលពី ថ្ងៃទី ០៣ ខែកុម្ភ : ឆ្នាំ ២០១៩ នៅ ចំណុច ក្នុង តូប លក់ ទូរសព្ទ័ ដៃ xxunk,0,0,0.07,0.94
xxbos សូម បើកបរ ដោយ មាន ការប្រុងប្រយ័ត្ន ! បើក រថយន្ត ធ្លាក់ ផ្លូវ បណ្ដាលឲ្យ xxunk របួស ធ្ងន់ ០៤ នាក់ - xxup tvfb ត្បូងឃ្មុំ : មាន ករណី គ្រោះថ្នាក់ ចរាចរណ៍ នៅលើ xxunk ជាតិ លេខ ៧ ចន្លោះ បង្គោល គីឡូម៉ែត្រ លេខ ១៣៥ - ១៣៦ ត្រង់ ចំណុច មុខ ផ្ទះ ឈ្មោះ ឡោ នី ភេទ ប្រុស ស្ថិត នៅ ភូមិ ម្រាម ទាក ឃុំ xxunk ទី ២ ស្រុក ត្បូងឃ្មុំ ខេត្ត ត្បូងឃ្មុំ នៅ វេលា ម៉ោង ០០ និង ២០ នាទី រំលង អធ្រាត្រ ឈានចូល,1,1,0.05,0.95
xxbos សូម xxunk ! រថយន្ត កុង xxunk ដែក បំពង់ទីប បើក សុខ ៗ របូត ជ្រុះ ធ្លាក់ បុក ចំ អ្នក ជិះ ម៉ូតូ ពីក្រោយ បណ្តាលឱ្យ រងរបួស ធ្ងន់ ( ស្វាយរៀង ) ៖ រថយន្ត កុង xxunk គ្រឿង ខណៈ កំពុង បើកបរ លើ ផ្លូវ ជាតិ លេខ ១ សុខ ៗ ស្រាប់តែ របូត ធ្លាក់ ដែក បំពង់ទីប មួយ ដុំ ធំ ពីលើ រថយន្ត បណ្ដាលឱ្យ ត្រូវ ចំ អ្នក ជិះ ម៉ូតូ ម្នាក់ រងរបួស ធ្ងន់ 

In [0]:
# looking at mislabel records
str1 = 'ប្រកាស ថ្ងៃ ចូល បម្រើ កងទ័ព' #label=0, index df[76] train_df
str2 = 'ត្រូវ សមត្ថកិច្ច ប៉ុស្តិ៍ នគរបាល ទំនប់ ទឹក' #0 index 246 train_df
str3 ='ដោយ ខ្សែ រថភ្លើង' # ពី ខេត្ត បន្ទាយមានជ័យ' #1 index: 140
str98 = 'រថយន្ត បើក បុក រ៉ឺម៉ក' # ម៉ូតូ អេត ចាយ'
str99 = 'ម៉ូតូ អេត ចាយ' #'អេត ចាយ' #24,29 0 but should be 1, 33 correctly label 1 but model guesses as 0
#train_df.at[76, 'label'] = 0
#train_df.at[246, 'label'] = 0
#train_df.at[140, 'label'] = 1
train_df[75:76]
train_df[140:141]
#train_df[246:247]
#train_df[train_df['text'].str.contains(str1)]
#df[df['text'].str.contains(str99)]
#print(train_df[22:23]['text'])
df.loc[29,'text'] 

'សមត្ថកិច្ច ស្ទូច រថយន្ត កុងតឺន័រ ដើម្បី យក សព ចេញមក ក្រៅ  ខេត្ត កំពង់ឆ្នាំង  ៖  បុរស វ័យ ចំណាស់ ម្នាក់ បាន ស្លាប់ ភ្លាម ៗ  ជាប់ ក្រោម រថយន្ត  ជាមួយ រ៉ឺម៉ក ម៉ូតូ របស់ គាត់  ខណៈ ដែល មាន រថយន្ត ដឹក កុងតឺន័រ មួយ គ្រឿង បាន បើក បុក ពីក្រោយ  ហើយ ក្រឡាប់ សង្កត់ ពីលើ បុរស រងគ្រោះ  ក្នុង ទិសដៅ ស្រប គ្នា ។ ហេតុការណ៍ គ្រោះថ្នាក់ ចរាចរណ៍ ដ៏ រន្ធត់ នេះ បាន កើតឡើង កាលពី ម៉ោង ៥ និង  ៣០ នាទី ទៀប ភ្លឺ ថ្ងៃទី ២២  ខែមករា  ឆ្នាំ ២០១៩  នៅ ចន្លោះ គីឡូម៉ែត្រ លេខ ៧០  និង ៧១  លើ កំណាត់ ផ្លូវ ជាតិ លេខ ៥  នៅ ចំណុច ត្រពាំង ត្រី រ៉ស់  ស្ថិត ក្នុង ភូមិ រនាម ទទឹង  ឃុំ ទឹកហូត  ស្រុក រលាប្អៀរ  ខេត្ត កំពង់ឆ្នាំង  ។ សមត្ថកិច្ច នគរបាល ចរាចរណ៍  នៅ កន្លែង កើត ហេតុ បាន អោយ ដឹង ថា  មុនពេល កើត ហេតុ  មាន រថយន្ត កុងតឺន័រ មួយ គ្រឿង ម៉ាក ហ្វុ៊យ សូ  ពណ៌ ស  ពាក់ ផ្លាកលេខ  ចង្កូត ស្តាំ  ភ្នំពេញ  PP-2307  បើកបរ ក្នុង ទិសដៅ ជើង ទៅ ត្បូង  លុះ ពេល មកដល់ ចំណុច កើត ហេតុ ក៏ បាន បុក ម៉ូតូ សង់ កូរ៉េ មួយ គ្រឿង មាន សណ្តោង រ៉ឺម៉ក ដឹក អេត ចាយ  ក្នុង ទិសដៅ ស្រប គ្នា  ហើយ រថយន្ត កុងតឺន័រ បាន ក្រឡាប់  និង កិន ពីលើ រ៉ឺម៉ក ម៉ូតូ  ព្រមទាំង អ្នកបើកបរ រ

In [0]:
# correct mislabels
df.at[76, 'label'] = 0
df.at[246, 'label'] = 0
df.at[140, 'label'] = 1
train_df.at[76, 'label'] = 0
train_df.at[246, 'label'] = 0
train_df.at[140, 'label'] = 1
train_df.at[22, 'label'] = 1
train_df.at[29, 'label'] = 1

In [0]:
learn_c.predict('កម្មករ បើក រថយន្ត ចេញ ពី លាង បុក រុញ គូទ')

(Category 0, tensor(0), tensor([0.6432, 0.3568]))

In [0]:
learn_c.predict('មក ស្គាល់ រុក្ខជាតិ ម្យ៉ាង នៅក្នុង សមុទ្រ')

(Category 0, tensor(0), tensor([9.9994e-01, 6.2050e-05]))

In [0]:
learn_c.predict('គ្រោះថ្នាក់ ចរាចរ រវាង រថយន្ត ធុន ធំ និង ម៉ូតូ បណ្តាលឲ្យ បុរស ម្នាក់ ស្លាប់ ក្រោម រថយន្ត')

(Category 1, tensor(1), tensor([0.0022, 0.9978]))

In [0]:
learn_c.predict('គ្រោះថ្នាក់ ចរាចរ រវាង រថយន្ត ធុន ធំ និង ម៉ូតូ បណ្តាលឲ្យ បុរស ម្នាក់ ស្លាប់ ក្រោម រថយន្ត យ៉ាង អាណោច មាន ករណី គ្រោះថ្នាក់ ចរាចរ យ៉ាង រន្ធត់ រវាង ')

(Category 1, tensor(1), tensor([1.1524e-05, 9.9999e-01]))

In [0]:
learn_c.predict('បណ្តាលឲ្យ ស្លាប់ មនុស្ស ស្រី ម្នាក់ និង មនុស្ស ប្រុស ស្រី ៤ នាក់ របួស ធ្ងន់')

(Category 0, tensor(0), tensor([0.9057, 0.0943]))

In [0]:
learn_c.predict('គ្រោះថ្នាក់ ចរាចរ រថយន្ត បែន ដឹក គ្រួស បើក បុក រះ រថយន្ត និង ម៉ូតូ ជាច្រើន គ្រឿង រួច រ៉េ បុក ចូល ផ្ទះ បណ្តាលឲ្យ ស្លាប់ មនុស្ស ស្រី ម្នាក់ និង មនុស្ស ប្រុស ស្រី ៤ នាក់ របួស ធ្ងន់')

(Category 1, tensor(1), tensor([0.0047, 0.9953]))

In [0]:
path

PosixPath('kmwiki_data')

In [0]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load(f'{lang}clas', purge=False);

In [0]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)
# best: (tensor(0.9750), tensor(0.9677)) -- first time 
#without crf data
#(tensor(0.9375), tensor(0.9206))

(tensor(0.9750), tensor(0.9677))

## Backward model

### Create backward pretrain model

Run copy segment zip file from gdrive [here](#scrollTo=IAPwj0tBu-QB)

In [0]:
data_path = Config.data_path()
lang = 'km'
name = f'{lang}wiki'
path = data_path/name
lm_fns = [f'{lang}_wt_bwd', f'{lang}_wt_vocab_bwd']

In [0]:
#should set to kmwiki_data --segemented
segpath = Path('kmwiki_data')
print(segpath)

kmwiki_data


In [0]:
#set backwards=True
data = (TextList.from_folder(segpath) # change from dest to segpath
            .split_by_rand_pct(0.2, seed=1)# split 0.1 to 0.2
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1, backwards=True))

data.save(f'{lang}_databunch_bwd')
len(data.vocab.itos),len(data.train_ds)

(50912, 2029)

In [0]:
!ls -alh kmwiki_data/'km_databunch_bwd'
!cp kmwiki_data/km_databunch_bwd drive/'My Drive'/khmer_ulmfit/

-rw-r--r-- 1 root root 73M Jan 31 03:22 kmwiki_data/km_databunch_bwd


In [0]:
#copy from gdrive
!mkdir kmwiki_data
!cp drive/'My Drive'/khmer_ulmfit/km_databunch_bwd kmwiki_data/

mkdir: cannot create directory ‘kmwiki_data’: File exists


In [0]:
data = load_data(segpath, f'{lang}_databunch_bwd', bs=bs, backwards=True)

In [0]:
# create learn
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [0]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7)) # should be 10 cycle

epoch,train_loss,valid_loss,accuracy,time
0,4.604825,5.202893,0.217026,12:58
1,4.097242,4.979578,0.232338,13:03
2,4.402261,4.874829,0.239404,13:05
3,4.306592,4.736785,0.252603,13:05
4,3.949404,4.598291,0.268997,13:05
5,4.141735,4.424586,0.287725,13:05
6,3.548918,4.251736,0.310711,13:05
7,3.722378,4.103264,0.331926,13:07
8,3.325685,4.034877,0.343804,13:07
9,3.210387,4.020954,0.346292,13:06


In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [0]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))
print("File save to:", mdl_path)

File save to: /root/.fastai/data/kmwiki/models


In [0]:
# copy dic and pkl file to gdrive
!cp {path}/models/km_wt* drive/'My Drive'/khmer_ulmfit/models
!ls drive/'My Drive'/khmer_ulmfit/models

kmclas.pth	      km_wt_bwd.pth  km_wt_test.pth
kmfine_tuned_enc.pth  km_wt.pkl.pth  km_wt_vocab_bwd.pkl
kmfine_tuned.pth      km_wt.pth      km_wt_vocab.pkl


In [0]:
!ls -alh drive/'My Drive'/khmer_ulmfit/models

total 2.0G
-rw------- 1 root root 208M Jan 14 00:34 kmclas.pth
-rw------- 1 root root 127M Jan 15 00:42 kmfine_tuned_enc.pth
-rw------- 1 root root 294M Jan 15 00:42 kmfine_tuned.pth
-rw------- 1 root root 156M Jan 31 06:05 km_wt_bwd.pth
-rw------- 1 root root 391M Jan 15 21:23 km_wt.pkl.pth
-rw------- 1 root root 391M Jan 15 21:24 km_wt.pth
-rw------- 1 root root 391M Jan 15 21:25 km_wt_test.pth
-rw------- 1 root root 1.3M Jan 31 06:05 km_wt_vocab_bwd.pkl
-rw------- 1 root root 1.4M Oct  9 04:52 km_wt_vocab.pkl


### Backward Language Model 

Load accident data from gdrive [here](#scrollTo=ExeT1IBFxlEf).

In [0]:
df.head() # make sure it has "label" column

,<built-in function id>,text,cat,label
0,5279,ចាប់ពី ថ្ងៃ នេះ តទៅ YouTube លែង ឲ្យ បង្ហោះ វ...,no_accident,0
1,5831,មក ស្គាល់ រុក្ខជាតិ ម្យ៉ាង នៅក្នុង សមុទ្រ កម្...,no_accident,0
2,6385,មក មើល កម្មករ ខ្សាច់ សាច់ដុំ ធំ ៗ នៅ កាមេរ៉ូន...,no_accident,0
3,7452,ដោយសារតែ បើកបរ លឿន ជិត ៗ គ្នា ពេក រថយន្ត ៦ គ្...,accident,1
4,7474,គ្មាន ទៀត ទេ រថយន្តក្រុង ប្រើ សាំង ! សៀង ហៃ ក...,no_accident,0


In [0]:
print(path) #/root/.fastai/data/kmwiki
!mkdir {path}/models
print(bs)
!cp drive/'My Drive'/khmer_ulmfit/models/km_wt_*bwd.* {path}/models
!ls -alh {path}/models

/root/.fastai/data/kmwiki
mkdir: cannot create directory ‘/root/.fastai/data/kmwiki/models’: File exists
48
total 157M
drwxr-xr-x 2 root root 4.0K Jan 31 14:54 .
drwxr-xr-x 3 root root 4.0K Jan 31 14:54 ..
-rw------- 1 root root 156M Jan 31 15:03 km_wt_bwd.pth
-rw------- 1 root root 1.3M Jan 31 15:03 km_wt_vocab_bwd.pkl


In [0]:
lm_fns = [f'km_wt_bwd', f'km_wt_vocab_bwd']

In [0]:
data_lm = (TextList.from_df(df, path, cols='text')
    .split_by_rand_pct(0.2, seed=1)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1, backwards=True))
# need lm_fns pretrain files (km_wt_bwd.pth, km_wt_vocab_bwd.pkl without the extention)
learn_lm = language_model_learner(data_lm, AWD_LSTM, config={**awd_lstm_lm_config, 'n_hid': 1152},
                                  pretrained_fnames=lm_fns, drop_mult=1.0)

In [0]:
lr = 1e-3
lr *= bs/48

In [0]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.558992,4.484405,0.237054,00:03
1,5.029195,4.271654,0.257381,00:03


In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))
#0	4.678388	4.229544	0.262500	00:04
#7	4.090632	3.793665	0.314970	00:04
#rerun again
#0	4.050783	3.784492	0.315804	00:04
#7	3.708633	3.576459	0.342292	00:04

epoch,train_loss,valid_loss,accuracy,time
0,4.050783,3.784492,0.315804,00:04
1,4.029990,3.740085,0.321607,00:04
2,3.978343,3.686088,0.327321,00:04
3,3.911218,3.639030,0.335298,00:04
4,3.846062,3.606670,0.339286,00:04
5,3.781395,3.585652,0.341131,00:04
6,3.733664,3.575690,0.342470,00:04
7,3.708633,3.576459,0.342292,00:04


In [0]:
learn_lm.save(f'{lang}fine_tuned_bwd')
learn_lm.save_encoder(f'{lang}fine_tuned_enc_bwd')

In [0]:
!find /root/.fastai/data/ -name *fine_tuned_bwd*

/root/.fastai/data/kmwiki/models/kmfine_tuned_bwd.pth


In [0]:
!ls /root/.fastai/data/kmwiki/models/

kmfine_tuned_bwd.pth	  km_wt_bwd.pth
kmfine_tuned_enc_bwd.pth  km_wt_vocab_bwd.pkl


#### Backward classifier

In [0]:
print(path)

/root/.fastai/data/kmwiki


In [0]:
# need df from above
data_clas = (TextList.from_df(df, path, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(0.1, seed=40)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1, backwards=True))

data_clas.save(f'{lang}_textlist_class_bwd') # todo add pth ext, so don't need rename, still need move
print(path)

#rerun with 0.1 split get warning in fit_one_cycle
#/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
#  average, "true nor predicted", 'F-score is', len(true_sum)

/root/.fastai/data/kmwiki


In [0]:
!cp /root/.fastai/data/kmwiki/km_textlist_class_bwd /root/.fastai/data/kmwiki/models/km_textlist_class_bwd.pth
!ls /root/.fastai/data/kmwiki/models/*bwd*

/root/.fastai/data/kmwiki/models/kmclas_bwd.pth
/root/.fastai/data/kmwiki/models/kmfine_tuned_bwd.pth
/root/.fastai/data/kmwiki/models/kmfine_tuned_enc_bwd.pth
/root/.fastai/data/kmwiki/models/km_textlist_class_bwd.pth
/root/.fastai/data/kmwiki/models/km_wt_bwd.pth
/root/.fastai/data/kmwiki/models/km_wt_vocab_bwd.pkl


In [0]:
data_clas = load_data(path, 'models/km_textlist_class_bwd.pth', bs=bs, num_workers=1, backwards=True)

In [0]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [0]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc_bwd')
learn_c.freeze()

In [0]:
lr=2e-2
lr *= bs/48

In [0]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))
#0	0.447624	0.559497	0.710000	0.641640	00:02
#1	0.348235	0.439373	0.780000	0.438857	00:02

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.369247,0.395085,0.740000,0.128000,00:02
1,0.267210,0.312172,0.800000,0.426667,00:02


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [0]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.234500,0.184478,0.940000,0.860690,00:03
1,0.200544,0.185770,0.960000,0.886154,00:02


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [0]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))
#0	0.166710	0.149070	0.940000	0.875294	00:04
#1	0.145192	0.150277	0.940000	0.873333	00:04

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.155800,0.115062,0.960000,0.891429,00:04
1,0.130948,0.098630,0.960000,0.891429,00:04


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [0]:
learn_c.unfreeze()
learn_c.fit_one_cycle(10, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7)) # get at most 0.975 --with 0.2 testset max 0.937

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.088257,0.104818,0.980000,0.926897,00:05
1,0.092945,0.130413,0.960000,0.896000,00:05
2,0.097039,0.125772,0.980000,0.926897,00:05
3,0.090649,0.131307,0.980000,0.926897,00:05
4,0.087074,0.139555,0.980000,0.926897,00:05
5,0.082950,0.122387,0.980000,0.926897,00:05
6,0.078888,0.125744,0.980000,0.926897,00:05
7,0.078793,0.127568,0.980000,0.926897,00:05
8,0.074976,0.134815,0.980000,0.926897,00:05
9,0.072815,0.132644,0.980000,0.926897,00:05


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMe

In [0]:
learn_c.unfreeze()
learn_c.fit_one_cycle(10, slice(lr/10/(2.6**4),lr/10), moms=(0.5,0.4)) # lower the momentum and run several time get 100%
# with 0.2 test 
#0	0.059526	0.175025	0.950000	0.904762	00:05
#9	0.050669	0.187786	0.960000	0.925333	00:05
# with 0.1 test
#0	0.058575	0.124074	0.980000	0.926897	00:05
#9	0.046329	0.129178	0.980000	0.926897	00:05

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.058575,0.124074,0.980000,0.926897,00:05
1,0.053043,0.125374,0.980000,0.926897,00:05
2,0.059490,0.132165,0.980000,0.926897,00:05
3,0.057090,0.114599,0.980000,0.926897,00:05
4,0.061594,0.138356,0.980000,0.926897,00:05
5,0.057663,0.122756,0.980000,0.926897,00:05
6,0.055327,0.126418,0.980000,0.926897,00:05
7,0.051005,0.133922,0.980000,0.926897,00:05
8,0.047676,0.130120,0.980000,0.926897,00:05
9,0.046329,0.129178,0.980000,0.926897,00:05


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMe

In [0]:
learn_c.save(f'{lang}clas_bwd')

### Load BWD model and predict

In [0]:
data_clas_bwd = load_data(path, f'{lang}_textlist_class_bwd', bs=bs, num_workers=1, backwards=True)
learn_c_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c_bwd.load(f'{lang}clas_bwd', purge=False);

In [0]:
#preds_b,targs_b = learn_c_bwd.get_preds(ordered=True)
preds_b,targs_b = learn_c_bwd.get_preds(ordered=True)
accuracy(preds_b,targs_b),f1(preds_b,targs_b)

(tensor(0.9800), tensor(0.9655))

## Ensemble

In [0]:
preds_avg = (preds*0.35+preds_b*0.65)/2

In [0]:
print(preds[0:9])
print(preds_b[0:9])

tensor([[1.0000e+00, 4.7293e-06],
        [9.9863e-01, 1.3670e-03],
        [9.9563e-01, 4.3653e-03],
        [9.9975e-01, 2.5266e-04],
        [1.0000e+00, 5.1229e-07],
        [1.2603e-07, 1.0000e+00],
        [3.2358e-10, 1.0000e+00],
        [1.9282e-11, 1.0000e+00],
        [9.9999e-01, 7.7065e-06]])
tensor([[9.9988e-01, 1.1774e-04],
        [9.9905e-01, 9.4546e-04],
        [2.2593e-01, 7.7407e-01],
        [9.9664e-01, 3.3635e-03],
        [9.9998e-01, 2.3278e-05],
        [7.4986e-06, 9.9999e-01],
        [1.8699e-07, 1.0000e+00],
        [2.2685e-08, 1.0000e+00],
        [9.9992e-01, 8.0455e-05]])


In [0]:
accuracy(preds_avg,targs_b),f1(preds_avg,targs_b)

(tensor(1.), tensor(1.))

### Test using bwd model

In [0]:
learn_c_bwd.predict('កម្មករ បើក រថយន្ត ចេញ ពី លាង បុក រុញ គូទ')

(Category 0, tensor(0), tensor([1.0000e+00, 2.0104e-06]))

In [0]:
learn_c_bwd.predict('មក ស្គាល់ រុក្ខជាតិ ម្យ៉ាង នៅក្នុង សមុទ្រ')

(Category 0, tensor(0), tensor([1.0000e+00, 2.9251e-06]))

In [0]:
learn_c_bwd.predict('គ្រោះថ្នាក់ ចរាចរ រវាង រថយន្ត ធុន ធំ និង ម៉ូតូ បណ្តាលឲ្យ បុរស ម្នាក់ ស្លាប់ ក្រោម រថយន្ត')

(Category 1, tensor(1), tensor([3.7043e-07, 1.0000e+00]))

In [0]:
learn_c_bwd.predict('គ្រោះថ្នាក់ ចរាចរ រវាង រថយន្ត ធុន ធំ និង ម៉ូតូ បណ្តាលឲ្យ បុរស ម្នាក់ ស្លាប់ ក្រោម រថយន្ត យ៉ាង អាណោច មាន ករណី គ្រោះថ្នាក់ ចរាចរ យ៉ាង រន្ធត់ រវាង ')

(Category 1, tensor(1), tensor([6.8543e-06, 9.9999e-01]))

In [0]:
learn_c_bwd.predict('បណ្តាលឲ្យ ស្លាប់ មនុស្ស ស្រី ម្នាក់ និង មនុស្ស ប្រុស ស្រី ៤ នាក់ របួស ធ្ងន់')

(Category 1, tensor(1), tensor([0.0044, 0.9956]))

In [0]:
learn_c_bwd.predict('គ្រោះថ្នាក់ ចរាចរ រថយន្ត បែន ដឹក គ្រួស បើក បុក ')

(Category 0, tensor(0), tensor([0.7569, 0.2431]))